# Search through gnps libraries to select dataset

Starting from gnps public libraries (speclibs folders, version from 5 July 2019), we here create one large MGF file with all spectra.
In addition Smiles or Inchi's are added from metadata file.

## Import libraries and folders

In [2]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
import os
import sys
sys.path.insert(0, os.path.dirname(os.getcwd()))

# Locations (import paths from config.py)
from config import ROOT, PATH_SPEC2VEC, PATH_MS_DATA, PATH_SAVE_MODEL, PATH_COMPUTED, PATH_OUTPUT

sys.path.insert(0, PATH_SPEC2VEC)

import MS_functions #import Spec2Vec functions related to MS spectra

## Importing additional metadata

In [37]:
json_file = os.path.join(PATH_MS_DATA, 'gnpslibraryjson_filtered.json')

with open(json_file, 'rb') as f:
    data = json.load(f)

In [4]:
print("Metadata available for", len(data), "spectra.")

Metadata available for 77253 spectra.


In [38]:
gnps_data = {}
for data_entry in data:
    if data_entry['GNPSID'] in gnps_data:
        print('duplicate!')
    gnps_data[data_entry['GNPSID']] = data_entry

## Briefly inspect metadata

In [6]:
keys = []
for key in gnps_data.keys():
    keys.append(key)

In [7]:
keys[0]

'CCMSLIB00000001548'

In [8]:
gnps_data[keys[0]]

{'COMPOUND_INCHIKEY': 'KNGPFNUOXXLKCN-ZNCJFREWSA-N',
 'COMPOUND_INCHI': 'InChI=1S/C45H73N5O10S3/c1-14-17-24(6)34(52)26(8)37-25(7)30(58-13)18-31-46-29(19-61-31)39-49-45(12,21-62-39)43-50-44(11,20-63-43)42(57)48-32(22(4)15-2)35(53)27(9)40(55)59-36(23(5)16-3)38(54)47-33(28(10)51)41(56)60-37/h19,22-28,30,32-37,51-53H,14-18,20-21H2,1-13H3,(H,47,54)(H,48,57)/t22-,23-,24+,25-,26-,27+,28+,30-,32-,33-,34-,35-,36-,37-,44+,45+/m0/s1',
 'LIBRARY_QUALITY': '1',
 'GNPSID': 'CCMSLIB00000001548',
 'COMPOUND_NAME': 'Hoiamide B',
 'COMPOUND_SMILES': 'CCC[C@@H](C)[C@@H]([C@H](C)[C@@H]1[C@H]([C@H](Cc2nc(cs2)C3=N[C@](CS3)(C4=N[C@](CS4)(C(=O)N[C@H]([C@H]([C@H](C(=O)O[C@H](C(=O)N[C@H](C(=O)O1)[C@@H](C)O)[C@@H](C)CC)C)O)[C@@H](C)CC)C)C)OC)C)O'}

## Define functions to match spectra with additional metadata json file

In [9]:
def check_spectrum_lines(spectrum):
    """ Check if desired metadata is present for spectrum.
    """
    spec_id = None
    smiles = None
    
    for l in spectrum:
        if l.startswith('SPECTRUMID'):
            spec_id = l.strip().split('=')[1]
        if l.startswith('SMILES'):
            smiles = l.strip().split('=')[1]
    
    if spec_id in gnps_data:
        return True
    else:
        return False
    if smiles is not None \
        and smiles.strip() != 'N/A' \
        and smiles.strip() != 'NA'\
        and len(smiles) > 0 \
        and smiles.strip() != 'InChI':
        return True

    return False

def augment_spectrum(spectrum):
    """ Add metadata to spectrum (taken from json file, see above).
    """
    for l in spectrum:
        if l.startswith('SPECTRUMID'):
            spectrum_id = l.strip().split('=')[1]
                
    new_spectrum = []
    for l in spectrum:
        if l.startswith('BEGIN ION'):
            new_spectrum.append(l)
            new_spectrum.append('TITLE=%s\n' % gnps_data[spectrum_id]['COMPOUND_NAME'])
            new_spectrum.append('LIBRARYQUALITY=%s\n' % gnps_data[spectrum_id]['LIBRARY_QUALITY'])
            new_spectrum.append('SMILES=%s\n' % gnps_data[spectrum_id]['COMPOUND_SMILES'])
            new_spectrum.append('INCHIKEY=%s\n' % gnps_data[spectrum_id]['COMPOUND_INCHIKEY'])
            new_spectrum.append('INCHI=%s\n' % gnps_data[spectrum_id]['COMPOUND_INCHI'])
            continue
            
        if l.startswith('INCHIKEY'):
            continue
        elif l.startswith('SMILES'):
            continue
        elif l.startswith('INCHI'):
            continue
        elif l.startswith('LIBRARYQUALITY'):
            continue
        elif l.startswith('TITLE'):
            continue
            
        new_spectrum.append(l)

    return new_spectrum

def match_spectra_in_file(filename, matching=True):
    """ Collect spectra in file (line by line). Check if matching metadata is available.
    
    matching: bool
        If matching = True than only include spectra for which desired metadata is present.
        For matching = False, include all spectra.
    """
    matched_spectra = []
    with open(filename) as f:
        for line in f.readlines():
            if line.startswith('BEGIN ION'):
                # Fresh spectrum
                spectrum = []
            spectrum.append(line)
            
            if line.startswith('END ION'):
                
                if check_spectrum_lines(spectrum): # Check if we have structural info
                    try:
                        spectrum = augment_spectrum(spectrum)
                    except KeyError as e:
                        pass
                    matched_spectra.append(spectrum)
                else: 
                    if not matching: # then include all spectra
                        matched_spectra.append(spectrum)
    return matched_spectra

## Create MGF for all spectra (also non matched ones)

In [10]:
path = os.path.join(PATH_MS_DATA, 'speclibs')
collected_spectra = []
for d in os.listdir(path):
    for f in os.listdir(os.path.join(path, d)):
        if not f.endswith('.mgf'):
            continue
        sp = match_spectra_in_file(os.path.join(path, d, f), matching=False)
        collected_spectra.extend(sp)
        print(f, len(collected_spectra))

BILELIB19.mgf 177
CASMI.mgf 745
DEREPLICATOR_IDENTIFIED_LIBRARY.mgf 1124
GNPS-COLLECTIONS-MISC.mgf 1170
GNPS-COLLECTIONS-PESTICIDES-NEGATIVE.mgf 1246
GNPS-COLLECTIONS-PESTICIDES-POSITIVE.mgf 1899
GNPS-EMBL-MCF.mgf 2484
GNPS-FAULKNERLEGACY.mgf 2611
GNPS-LIBRARY.mgf 7394
GNPS-NIH-CLINICALCOLLECTION1.mgf 7771
GNPS-NIH-CLINICALCOLLECTION2.mgf 7966
GNPS-NIH-NATURALPRODUCTSLIBRARY.mgf 9233
GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_NEGATIVE.mgf 11096
GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_POSITIVE.mgf 16892
GNPS-NIH-SMALLMOLECULEPHARMACOLOGICALLYACTIVE.mgf 18352
GNPS-NIST14-MATCHES.mgf 24115
GNPS-PRESTWICKPHYTOCHEM.mgf 24258
GNPS-SELLECKCHEM-FDA-PART1.mgf 26646
GNPS-SELLECKCHEM-FDA-PART2.mgf 27302
HMDB.mgf 29537
LDB_NEGATIVE.mgf 29763
LDB_POSITIVE.mgf 29846
MASSBANK.mgf 41845
MASSBANKEU.mgf 43337
MIADB.mgf 43509
MMV_NEGATIVE.mgf 43556
MMV_POSITIVE.mgf 43666
MONA.mgf 92907
PNNL-LIPIDS-NEGATIVE.mgf 109049
PNNL-LIPIDS-POSITIVE.mgf 139631
RESPECT.mgf 146743
SUMNER.mgf 147647


In [11]:
print("Collected", len(collected_spectra), "spectra from: \n", path)

Collected 147647 spectra from: 
 C:\OneDrive - Netherlands eScience Center\Project_Wageningen_iOMEGA\Data\GNPS_all\speclibs\


# Convert smiles to inchi where necessary

### Make inchi strings more homogeneous in style

In [29]:
def cleanup_inchi(spectrum, rescue_smiles = True):
    """
    Read spectrum, look for inchi. Then:
    1) Make line with inchi homogeneously looking like: 'INCHI="InChI=..."'
    2) if rescue_smiles is True then try to detect inchi that are actually smiles 
    and convert to proper inchi (using openbabel based function from MS_functions.py).
    """
  
    spectrum_new = []
    inchi = 'N/A'

    for l in spectrum:
        if l.startswith('INCHI='):
            inchi = l.strip().split('INCHI=')[1]
            
            if inchi is None \
                or inchi in ['N/A', 'NA', 0, '0', '""'] \
                or len(inchi) == 0:
                pass
            else:
                inchi = inchi.replace(" ", "") #remove empty spaces
                if inchi.split('InChI=')[-1][0] in ['C', 'c', 'O', 'N']:
                    if rescue_smiles:
                        # Try to 'rescue' given inchi which are actually smiles!
                        assumed_smile = inchi.split('InChI=')[-1].replace('"', '')
                        inchi = MS_functions.mol_converter(assumed_smile, "smi", "inchi")
                        print('new inchi:', inchi, assumed_smile)
                
                inchi = inchi.strip().split('InChI=')[-1]
                if inchi.endswith('"'):
                    inchi = '"InChI=' + inchi
                elif inchi.endswith('\n'):
                    inchi = '"InChI=' + inchi[:-2] + '"'
                elif inchi.endswith('\n"'):
                    inchi = '"InChI=' + inchi[:-3] + '"'
                else:
                    inchi = '"InChI=' + inchi + '"'
                l = 'INCHI=' + inchi + '\n'
        spectrum_new.append(l)
    return spectrum_new

In [30]:
for i, spectrum in enumerate(collected_spectra):      
    spectrum_new = cleanup_inchi(spectrum, rescue_smiles = True)
    collected_spectra[i] = spectrum_new 

new inchi: InChI=1S/C7H8N4O2/c1-10-5-4(8-3-9-5)6(12)11(2)7(10)13/h3H,1-2H3,(H,8,9)
 Cn(c1nc[nH]c1c(=O)n2C)c2=O
new inchi: InChI=1S/C42H60N6O11/c1-24(2)13-11-9-7-8-10-12-14-35(52)47(5)32(23-49)40(56)44-25(3)38(54)43-22-36(53)48(6)37-28-16-18-34(51)30(21-28)29-19-27(15-17-33(29)50)20-31(42(58)59)46-39(55)26(4)45-41(37)57/h15-19,21,24-26,31-32,37,49-51H,7-14,20,22-23H2,1-6H3,(H,43,54)(H,44,56)(H,45,57)(H,46,55)(H,58,59)
 CC1C(NC(C(O)=O)CC2=CC(C3=C(O)C=CC(C(N(C(CNC(C(NC(C(N(C(CCCCCCCCC(C)C)=O)C)CO)=O)C)=O)=O)C)C(N1)=O)=C3)=C(O)C=C2)=O
new inchi: InChI=1S/C87H139N15O25/c1-16-18-19-20-21-24-54(103)44-70(109)92-56(31-35-66(88)105)77(113)93-57(32-36-67(89)106)78(114)94-58(33-37-68(90)107)79(115)95-59(34-38-69(91)108)86(122)126-52(12)82(118)101-39-22-25-61(101)84(120)99(13)63(42-47(5)6)80(116)98-73-51(11)125-87(123)64(43-53-27-29-55(124-15)30-28-53)100(14)85(121)62-26-23-40-102(62)83(119)60(41-46(3)4)96-76(112)50(10)74(111)75(48(7)8)127-71(110)45-65(104)72(49(9)17-2)97-81(73)117/h27-30,46-52,54

new inchi: InChI=1S/C11H12N2O2/c12-9(11(14)15)5-7-6-13-10-4-2-1-3-8(7)10/h1-4,6,9,13H,5,12H2,(H,14,15)/t9-/m0/s1
 O=C(O)[C@@H](N)Cc1c[nH]c2c1cccc2


#### Insert save point here

## Search for spectra with missing smiles or inchi (and try to convert between those)
+ Create inchi for every entry that has smiles but no inchi
+ Create smiles for every entry that has inchi but no smiles
+ Create inchikey for every entry that has an inchi but no inchikey yet.

In [4]:
from openbabel import openbabel as ob
from openbabel import pybel

inchi_lst = []
inchikey_lst = []
smiles_lst = []

# An empirically found list of strings that occured in the dataset and represent empty entries
empty_entry_types = ['N/A', 'n/a', 'NA', 0, '0', '""', '', 'nodata', 
                     '"InChI=n/a"', '"InChI="', 'InChI=1S/N\n', '\t\r\n']

changes_to_spectra = []
for i, spectrum in enumerate(collected_spectra):
    # Show progress
    if i // 10000 == i/10000:
        print("running spectra no", i)
        
    smiles = 'N/A'
    inchi = 'N/A'
    inchikey = 'N/A'
    modified = False
    
    smiles_line = 0
    inchi_line = 0
    inchikey_line = 0
    inchiaux_line = 0
    for m, l in enumerate(spectrum):
        if l.startswith('SMILES='):
            smiles = l.strip().split('SMILES=')[1].replace(" ", "")
            smiles_line = m
        if l.startswith('INCHI='):
            inchi = l.strip().split('INCHI=')[1].replace(" ", "")
            inchi_line = m
        if l.startswith('INCHIKEY='):
            inchikey = l.strip().split('INCHIKEY=')[1].replace(" ", "")
            inchikey_line = m
        if l.startswith('INCHIAUX='):
            inchiaux_line = m

    # Add inchi line if not present        
    if inchi_line == 0: 
        spectrum.insert(smiles_line+1, 'INCHI=N/A\n')
        inchi_line = smiles_line+1
        
    # Add smiles line if not present        
    if smiles_line == 0: 
        spectrum.insert(inchi_line+1, 'SMILES=N/A\n')

    # Add inchikey line if not present
    if inchikey_line == 0: 
        if inchiaux_line == 0:
            spectrum.insert(inchi_line+1, 'INCHIKEY=N/A\n')
        else:
            spectrum[inchiaux_line] = 'INCHIKEY=N/A\n'           
        
    # Add to lists
    smiles_lst.append(smiles)
    inchi_lst.append(inchi)
    inchikey_lst.append(inchikey)
    
    # Look for cases with smiles BUT NO inchi 
    if inchi is None \
        or inchi in empty_entry_types \
        or len(inchi) < 12:
        if smiles is not None \
            and smiles not in empty_entry_types \
            and len(smiles) > 0:  
            for m, l in enumerate(spectrum):
                if l.startswith('INCHI='):
                    inchi = MS_functions.mol_converter(smiles, "smi", "inchi")
                    if inchi == '': 
                        inchi = MS_functions.mol_converter(smiles, "smy", "inchi")# test if smiley parser works
                        
                    #if inchi.endswith("\n"): # --> led to error!!
                    #    inchi = inchi[:-2]
                    l = 'INCHI=' + '"' + inchi+ '"\n'
                    spectrum[m] = l
                    modified = True
                    inchi_lst[-1] = inchi 
                    break #assuming there is only one line with inchi
                            
    # Look for cases with inchi BUT NO smiles
    if smiles is None \
        or smiles in empty_entry_types \
        or len(smiles) == 0:  
        if inchi is not None \
            and inchi not in empty_entry_types \
            and len(inchi) > 12 \
            and inchi.split('=')[1][0] == '1': # Filter out empty and defective inchi
            for m, l in enumerate(spectrum):
                if l.startswith('SMILES='):
                    smiles = MS_functions.mol_converter(inchi, "inchi", "smi")
                    l = 'SMILES=' + smiles
                    spectrum[m] = l
                    modified = True
                    smiles_lst[-1] = smiles
                    break #assuming there is only one line with smiles

    # Look for missing inchikeys
    if inchikey is None \
        or inchikey in empty_entry_types \
        or len(inchikey) == 0:
        if len(inchi) > 12:
            if inchi.split('=')[1][0] == '1': # Filter out empty and defective inchi
                found_inchikey = MS_functions.mol_converter(inchi, "inchi", "inchikey")

                for m, l in enumerate(spectrum):
                    if l.startswith('INCHIKEY='):
                        l = 'INCHIKEY=%s' % found_inchikey
                        spectrum[m] = l
                        modified = True
                        inchikey_lst[-1] = found_inchikey
            else:
                print('Suspicious inchi found for spectrum', i, ': ', inchi )

    # Replace spectrum with modified one                                 
    if modified:                                       
        collected_spectra[i] = spectrum                  
        changes_to_spectra.append((i, inchi_lst[-1], inchikey_lst[-1]))

running spectra no 0
Suspicious inchi found for spectrum 5418 :  "InChI=DOUMFZQKYFQNTF-WUTVXBCWSA-N"
Suspicious inchi found for spectrum 5419 :  "InChI=DOUMFZQKYFQNTF-WUTVXBCWSA-N"
Suspicious inchi found for spectrum 5420 :  "InChI=QGJZLNKBHJESQX-FZFNOLFKSA-N"
Suspicious inchi found for spectrum 5421 :  "InChI=WCGUUGGRBIKTOS-GPOJBZKASA-N"
Suspicious inchi found for spectrum 5422 :  "InChI=WCGUUGGRBIKTOS-GPOJBZKASA-N"
Suspicious inchi found for spectrum 7932 :  "InChI=Brc1c(NC2=NCCN2)ccc3nccnc13"
running spectra no 10000
running spectra no 20000
running spectra no 30000
running spectra no 40000
running spectra no 50000
running spectra no 60000
running spectra no 70000
running spectra no 80000
running spectra no 90000
running spectra no 100000
running spectra no 110000
running spectra no 120000
running spectra no 130000
running spectra no 140000


### Inspect number of found annotations

In [5]:
num_inchi = len([x for x in inchi_lst if x not in empty_entry_types])
print("Number of spectra with inchi:", num_inchi)

num_inchikey = len([x for x in inchikey_lst if x not in empty_entry_types])
print("Number of spectra with inchikey:", num_inchikey)

num_smiles = len([x for x in smiles_lst if x.replace(" ", "") not in empty_entry_types])
print("Number of spectra with smiles:", num_smiles)

Number of spectra with inchi: 87851
Number of spectra with inchikey: 87845
Number of spectra with smiles: 87414


In [6]:
"""
# Save data here
import pickle
filename = 'collected_spectra_after_1st_cleaning'

with open(filename, "wb") as output:
    pickle.dump(collected_spectra, output, pickle.HIGHEST_PROTOCOL);
"""

In [3]:
"""
import pickle
filename = 'collected_spectra_after_1st_cleaning'
collected_spectra = pickle.load(open(filename, "rb"))
"""

## Look for inchi that cause issues
Here we simply detect them by looking for pairs where we still (after the conversions done above) find spectra with given inchi but without smiles.

In [7]:
inchi_no_smiles = []
for i, smiles in enumerate(smiles_lst):
    if smiles in empty_entry_types:
        if inchi_lst[i] not in empty_entry_types:
            inchi_no_smiles.append(i)
            
len(inchi_no_smiles)

438

In [8]:
[inchi_lst[ID] + " smiles: " + smiles_lst[ID] + " inchikey: " + inchikey_lst[ID] for ID in inchi_no_smiles[:10]]

['"InChI=DOUMFZQKYFQNTF-WUTVXBCWSA-N" smiles:  inchikey: N/A',
 '"InChI=DOUMFZQKYFQNTF-WUTVXBCWSA-N" smiles:  inchikey: N/A',
 '"InChI=QGJZLNKBHJESQX-FZFNOLFKSA-N" smiles:  inchikey: N/A',
 '"InChI=WCGUUGGRBIKTOS-GPOJBZKASA-N" smiles:  inchikey: N/A',
 '"InChI=WCGUUGGRBIKTOS-GPOJBZKASA-N" smiles:  inchikey: N/A',
 '"InChI=Brc1c(NC2=NCCN2)ccc3nccnc13" smiles: N/A inchikey: N/A',
 '"InChI=1S/C12H10O4/c17512(14)161169(158(2)13)3410(7)11/h36H,12H3" smiles: \t\r\n inchikey: WRMXLMVKGKNPFN-UHFFFAOYSA-N',
 '"InChI=1S/C12H10O4/c17512(14)161169(158(2)13)3410(7)11/h36H,12H3" smiles: \t\r\n inchikey: WRMXLMVKGKNPFN-UHFFFAOYSA-N',
 '"InChI=1S/C12H10O4/c17512(14)161169(158(2)13)3410(7)11/h36H,12H3" smiles: \t\r\n inchikey: WRMXLMVKGKNPFN-UHFFFAOYSA-N',
 '"InChI=1S/C21H26N2O3/c12621(25)191510172014(13423516(13)2220)8923(17)1112(15)6718(19)24/h25,12,15,1719,22,24H,611H2,1H3" smiles: \t\r\n inchikey: YRSRYWBTPBXXPH-UHFFFAOYSA-N']

### More systematic search for inchi with issues

In [10]:
suspected_inchi_issues = []

for i, inchi in enumerate(inchi_lst):
    # Show progress
    if i // 10000 == i/10000:
        print("testing inchi for spectrum no", i)
        
    if inchi not in empty_entry_types:
        if inchi.split('=')[1][0] == '1': # Filter out empty and defective inchi
            # Check if inchi can be handled by openbabel
            smiles = MS_functions.mol_converter(inchi, "inchi", "smi")
            if smiles in empty_entry_types:
                suspected_inchi_issues.append((i, inchi))
        else:
            suspected_inchi_issues.append((i, inchi))

testing inchi for spectrum no 0
testing inchi for spectrum no 10000
testing inchi for spectrum no 20000
testing inchi for spectrum no 30000
testing inchi for spectrum no 40000
testing inchi for spectrum no 50000
testing inchi for spectrum no 60000
testing inchi for spectrum no 70000
testing inchi for spectrum no 80000
testing inchi for spectrum no 90000
testing inchi for spectrum no 100000
testing inchi for spectrum no 110000
testing inchi for spectrum no 120000
testing inchi for spectrum no 130000
testing inchi for spectrum no 140000


In [11]:
len(suspected_inchi_issues)

496

### Inspect the typically occuring issues

In [12]:
#[inchi_lst[ID] + " smiles: " + smiles_lst[ID] + " inchikey: " + inchikey_lst[ID] for (ID, _) in suspected_inchi_issues if not ID in inchi_no_smiles]
[inchi_lst[ID] + " inchikey: " + inchikey_lst[ID] for (ID, _) in suspected_inchi_issues[:20]]

['"InChI=1S/C28H32O14/c1-10-20(32)23(35)25(37)28(39-10)42-27-24(36)21(33)17(9-29)41-26(27)19-15(38-2)8-16-18(22(19)34)13(31)7-14(40-16)11-3-5-12(30)6-4-11/h3-8,10,17,20-21,23-30,32-37H,9H2,1-2H3/t10-,17-,20+,21?,23,24,25?,26?,27?,28?/m0/s1" inchikey: UINZSMIFSIQGBZ-CPTKKBKASA-N',
 '"InChI=1S/C21H20O12/c22-6-13-15(27)17(29)18(30)21(32-13)33-20-16(28)14-11(26)4-8(23)5-12(14)31-19(20)7-1-2-9(24)10(25)3-7/h1-5,13,15,17-18,21-27,29-30H,6H2/t13-,15-,17+,18-,21/m0/s1" inchikey: OVSQVDMCBVZWGM-XXRZTOGGSA-N',
 '"InChI=1S/C27H30O16/c1-8-17(32)20(35)22(37)26(40-8)39-7-15-18(33)21(36)23(38)27(42-15)43-25-19(34)16-13(31)5-10(28)6-14(16)41-24(25)9-2-3-11(29)12(30)4-9/h2-6,8,15,17-18,20-23,26-33,35-38H,7H2,1H3/t8-,15+,17-,18,20,21-,22,23,26,27-/m1/s1" inchikey: IKGXIBQEEMLURG-QYNJVRNOSA-N',
 '"InChI=1S/C22H22O10/c1-30-13-7-14-16(11(25)6-12(31-14)9-2-4-10(24)5-3-9)19(27)17(13)22-21(29)20(28)18(26)15(8-23)32-22/h2-7,15,18,20-24,26-29H,8H2,1H3/t15-,18-,20+,21-,22/m1/s1" inchikey: ABRULANJVVJLFI-PFGYYPEH

In [13]:
# Rescue inchi based on smiles
smiles = 'OC1=CC=C(C2=C(O[C@H]3O[C@@H](COC(/C=C/C4=CC=C(O)C=C4)=O)[C@@H](O)[C@@H](O)[C@@H]3O)C(C5=C(O)C=C(O)C=C5O2)=O)C=C1'
MS_functions.mol_converter(smiles, 'smi', 'inchi') not in empty_entry_types

True

## Frequently detected issues:

### 1) Inchi are actually inchikey
+ For some of those we at least have smiles --> can easily be rescued!
+ For some even the smiles is missing --> here we can only look for matches based on compound name (or formula) and inchikey.

### 2) Inchi with missing '-' characters
+ InChI=1S/C12H10O4/c1-7-5-12(14)16-11-6-9(15-8(2)13)3-4-10(7)11/h3-6H,1-2H3 #actual one (checked via PubChem)
+ InChI=1S/C12H10O4/c17512(14)161169(158(2)13)3410(7)11/h36H,12H3 #found in metadata

In total a few hundred inchi look like this. They mis the '-' in the inchi making them unreadable for further processing.  
In the following we will use pubchem to look for best matches for those inchi to replace the defective ones.

## Search pubchem for all spectra with problematic inchi

In [14]:
import pubchempy as pcp

In [16]:
results = pcp.get_compounds('4-methylumbelliferyl acetate', 'name')

inchi_pubchem = results[0].inchi
inchi_pubchem

'InChI=1S/C12H10O4/c1-7-5-12(14)16-11-6-9(15-8(2)13)3-4-10(7)11/h3-6H,1-2H3'

### Define necessary functions (other key functions are in MS_functions.py)

In [14]:
import pubchempy as pcp

def get_compound_name(spectrum):
    # Get compound name from MGF based spectra information
    # Expects list of lines from single spectrum from MGF file as input.
    
    compound_name = ''
        
    for l in spectrum:
        if l.startswith('TITLE='):
            if not l.strip().split('TITLE=')[1].startswith('Scan'):
                compound_name = l.strip().split('TITLE=')[1]

        if l.startswith('NAME=') and compound_name == '': # if no TITLE tag found take NAME
            compound_name = l.strip().split('NAME=')[1]

            # Remove adduct from name
            adduct_strings = ['[M+H-H2O]', 'M+H-H2O','[M+H]+','[M+H]-', '[M+H]', 'M+H+', 'M+H-', 'M+H']
            for adduct in adduct_strings:
                compound_name = compound_name.replace(adduct, '')
    
    return compound_name       


def replace_annotation_in_spectrum(ID, 
                                   inchi_new = None, 
                                   smiles_new = None,
                                   inchikey_new = None,
                                   ignore_entry_types = []):
    # Replace inchi in MGF based spectrum..
    spectrum_new = []

    for m, l in enumerate(collected_spectra[ID]):
        if l.startswith('INCHI='):
            if inchi_new is not None \
            and inchi_new not in ignore_entry_types:
                l = 'INCHI=' + inchi_new
        if l.startswith('INCHIKEY='):
            if inchikey_new is not None\
            and inchikey_new not in ignore_entry_types:
                l = 'INCHIKEY=' + inchikey_new
        if l.startswith('SMILES='):
            if smiles_new is not None\
            and smiles_new not in ignore_entry_types:
                l = 'SMILES=' + smiles_new
        spectrum_new.append(l)
    
    collected_spectra[ID] = spectrum_new
    

def inchi_is_inchikey(inchi):
    """ Test is inchi is actually an inchikey.
    To be used to detect wrongly labeled inchi.
    
    Output will be True if inchi has shape of an inchikey, and False otherwise.
    
    Args:
    --------
    inchi: str
        Input inchi to be tested.
    """
    inchi = inchi.replace('"', '').replace(' ', '')
    if inchi.startswith('InChI='):
        inchi_core = inchi.split('InChI=')[1]
    elif '=' in inchi:
        inchi_core = inchi.splot('=')[1]
    else:
        inchi_core = inchi
        
    if len(inchi_core) == 27 \
    and inchi_core[14] == inchi_core[25] == '-': # Assuming inchikey of form 'PFTAWBLQPZVEMU-DZGCQCFKSA-N'
        return True
    else:
        return False

In [15]:
inchi = "InChI=PFTAWBLQPZVEMU-DZGCQCFKSA-N"
inchi_is_inchikey(inchi)

True

## Loop through all spectra with problematic inchi and try to rescue them.
The key function here is MS_functions.find_pubchem_match.

### 1) Check if inchi is actually an inchikey
If it is -->  
**1a.** If smiles is present --> try to convert that into inchi.  
**1b.** If no smiles present (or 1a. failed) --> search matching full inchikey on Pubchem.  
**1c.** If 1a & 1b failed --> search for matching name + inchikey(first 14 letters only)  

### 2) (if not: -->) Searching for matching compound name + inchi
It will look for the compound name in Pubchem and then check if there is a good enough inchi match.
For this the first min_agreement parts of a pair of inchi needs to match (ignoring the '-' to account for the inchi defect.

### 3) (if not: -->) Searching the missing annotations for long cryptic names!
In those cases 1) didn't succeed. Usually that means that
+ The inchi is broken
+ AND the name is not found on pubchem (usually because it's long and cyptic!

A typical example would be: (4S,5Z,6S)-4-(2-methoxy-2-oxoethyl)-5-[2-[(E)-3-phenylprop-2-enoyl]oxyethylidene]-6-[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy-4H-pyran-3-carboxylic acid 

In those cases we extract the formula from the inchi and look for matches of that formula in Pubchem.  
Then we check if there is a good enough inchi match.
Again, this means that the first min_agreement parts of a pair of inchi needs to match (ignoring the '-' to account for the inchi defect.

In [16]:
replace_inchikey = True
replace_smiles = True
inchi_corrected = [] # To estimate how many rescues worked out...

print(40 * '--')
print("Searching Pubchem for matching compounds to correct defective inchi")
print(40 * '--')

for (ID, _) in suspected_inchi_issues:
    inchi = inchi_lst[ID]
    inchi_rescued = ''
    smiles_rescued = ''
    inchikey_rescued = ''
    corrected = False
    
    # Get compound name
    compound_name = get_compound_name(collected_spectra[ID])
    print(5 * '--' + compound_name + 5 * '--')  
    
    # 1) Check if inchi is inchikey:
    if inchi_is_inchikey(inchi):
        # Get inchikey from inchi
        inchi = inchi.replace('"', '').replace(' ', '')
        if inchi.startswith('InChI='):
            inchikey = inchi.split('InChI=')[1]
        elif '=' in inchi:
            inchikey = inchi.splot('=')[1]
        else:
            inchikey = inchi
        inchikey_rescued = inchikey # keep this one to later replace the entry in the spectra collection
        
        # 1a) smiles in metadata:
        if smiles_lst[ID] not in empty_entry_types:
            inchi_rescued = MS_functions.mol_converter(smiles_lst[ID], 'smi', 'inchi')
            if inchi_rescued in empty_entry_types:
                print('inchi could not be rescued from found smiles:', smiles_lst[ID])
            else:
                print('--> Found smiles was used to convert to inchi.')
                print("Inchi ( input ): " + inchi)
                print("Inchi (convert): " + inchi_rescued + "\n")
        
        # 1b) no smiles, or (1a) didn't succeed:
        if inchi_rescued in empty_entry_types:
            # Look for full inchikey match on Pubchem
            result = pcp.get_compounds(inchikey, 'inchikey')
            print("Found", len(result), "match(es) on Pubchem for inchikey", inchikey)
            
            inchi_rescued = result[0].inchi
            smiles_rescued = result[0].isomeric_smiles
            if inchi_rescued not in empty_entry_types:
                print("Inchi ( input ): " + inchi)
                print("Inchi (pubchem): " + inchi_rescued + "\n")
                
        # 1c) if still didn't succeed:
        if inchi_rescued in empty_entry_types \
        and compound_name != '':
            # Look for full matching name with matching first 14 letters of inchikey on Pubchem
            inchi_rescued, inchikey_rescued = MS_functions.find_pubchem_match(compound_name, 
                                                                           inchi = None, 
                                                                           inchikey = inchikey,
                                                                           mode = 'and',
                                                                           min_inchi_match = 3,
                                                                           min_inchikey_match = 1,
                                                                           name_search_depth = 10,
                                                                           formula_search = True,
                                                                           formula_search_depth = 25)
            
            if inchi_rescued is not None \
            and inchi_rescued not in empty_entry_types:
                # Also replace smiles
                if replace_smiles:
                    if len(inchi_rescued) > 12:
                        if inchi_rescued.split('=')[1][0] == '1': # Filter out empty and defective inchi
                            smiles_rescued = MS_functions.mol_converter(inchi_rescued, "inchi", "smi")
    
    else: # if inchi is no inchikey:
    # 2) Search on Pubchem based on compound name and (defective) inchi
    # + 3) If 2 didn't succeed, search on Pubchem based on formula and (defective) inchi    
        if compound_name != '':     
            inchi_rescued, inchikey_rescued = MS_functions.find_pubchem_match(compound_name, 
                                                                           inchi, 
                                                                           inchikey = None,
                                                                           mode = 'and',
                                                                           min_inchi_match = 3,
                                                                           min_inchikey_match = 1,
                                                                           name_search_depth = 10,
                                                                           formula_search = True,
                                                                           formula_search_depth = 25)

            if inchi_rescued is not None \
            and inchi_rescued not in empty_entry_types:
    
                #inchikey_rescued = None
                if replace_inchikey:
                    if inchikey_rescued is None \
                    or inchikey_rescued in empty_entry_types:
                        if len(inchi_rescued) > 12 \
                        and inchi_rescued.split('=')[1][0] == '1': # Filter out empty and defective inchi
                            inchikey_rescued = MS_functions.mol_converter(inchi_rescued, "inchi", "inchikey")

                smiles_rescued = None
                if replace_smiles:
                    if len(inchi_rescued) > 12:
                        if inchi_rescued.split('=')[1][0] == '1': # Filter out empty and defective inchi
                            smiles_rescued = MS_functions.mol_converter(inchi_rescued, "inchi", "smi")

    # ---------------------------------------------------
    # Finally, replace annotations if rescue was possible
    # ---------------------------------------------------
    if inchi_rescued is not None \
    and inchi_rescued not in empty_entry_types:
        inchi_corrected.append(1)
        
        # Replace inchi in spectrum collection
        print("Replaced current annotation with rescued data. \n")
        replace_annotation_in_spectrum(ID, 
                               inchi_new = inchi_rescued, 
                               smiles_new = smiles_rescued,
                               inchikey_new = inchikey_rescued,
                               ignore_entry_types = empty_entry_types) # only replace if not in empty types
    else:
        print("----- INCHI COULD NOT BE RESCUED FOR SPECTRUM:", ID ,"-----")
        inchi_corrected.append(0)

--------------------------------------------------------------------------------
Searching Pubchem for matching compounds to correct defective inchi
--------------------------------------------------------------------------------
----------2O-rhamnosyl-swertisin----------
Found at least 0 compounds of that name on pubchem.
Found at least 25 compounds with formula C28H32O14 on pubchem.
--> FOUND MATCHING COMPOUND ON PUBCHEM.
Inchi ( input ): "InChI=1S/C28H32O14/c1-10-20(32)23(35)25(37)28(39-10)42-27-24(36)21(33)17(9-29)41-26(27)19-15(38-2)8-16-18(22(19)34)13(31)7-14(40-16)11-3-5-12(30)6-4-11/h3-8,10,17,20-21,23-30,32-37H,9H2,1-2H3/t10-,17-,20+,21?,23,24,25?,26?,27?,28?/m0/s1"
Inchi (pubchem): "InChI=1S/C28H32O14/c1-10-20(32)23(35)25(37)28(39-10)42-27-24(36)21(33)17(9-29)41-26(27)19-15(38-2)8-16-18(22(19)34)13(31)7-14(40-16)11-3-5-12(30)6-4-11/h3-8,10,17,20-21,23-30,32-37H,9H2,1-2H3/t10-,17+,20-,21+,23+,24-,25+,26-,27+,28-/m0/s1"

Replaced current annotation with rescued data. 

--------

Found 1 match(es) on Pubchem for inchikey DOUMFZQKYFQNTF-WUTVXBCWSA-N
Inchi ( input ): InChI=DOUMFZQKYFQNTF-WUTVXBCWSA-N
Inchi (pubchem): InChI=1S/C18H16O8/c19-12-4-1-10(7-14(12)21)3-6-17(23)26-16(18(24)25)9-11-2-5-13(20)15(22)8-11/h1-8,16,19-22H,9H2,(H,24,25)/b6-3+/t16-/m1/s1

Replaced current annotation with rescued data. 

----------rosmarinic acid  ----------
Found 1 match(es) on Pubchem for inchikey DOUMFZQKYFQNTF-WUTVXBCWSA-N
Inchi ( input ): InChI=DOUMFZQKYFQNTF-WUTVXBCWSA-N
Inchi (pubchem): InChI=1S/C18H16O8/c19-12-4-1-10(7-14(12)21)3-6-17(23)26-16(18(24)25)9-11-2-5-13(20)15(22)8-11/h1-8,16,19-22H,9H2,(H,24,25)/b6-3+/t16-/m1/s1

Replaced current annotation with rescued data. 

----------Betulinic acid ----------
Found 1 match(es) on Pubchem for inchikey QGJZLNKBHJESQX-FZFNOLFKSA-N
Inchi ( input ): InChI=QGJZLNKBHJESQX-FZFNOLFKSA-N
Inchi (pubchem): InChI=1S/C30H48O3/c1-18(2)19-10-15-30(25(32)33)17-16-28(6)20(24(19)30)8-9-22-27(5)13-12-23(31)26(3,4)21(27)11-14-29(22,28)7/h19-24,3

Found at least 3 compounds of that name on pubchem.
--> FOUND MATCHING COMPOUND ON PUBCHEM.
Inchi ( input ): "InChI=1S/C27H42O4/c1-15-5-10-27(30-14-15)16(2)24-23(31-27)13-20-18-12-22(29)21-11-17(28)6-8-25(21,3)19(18)7-9-26(20,24)4/h15-21,23-24,28H,5-14H2,1-4H3/t15-,16+,17+,18-,19+,20+,21-,23+,24+,25-,26+,27-/m1/s"
Inchi (pubchem): "InChI=1S/C27H42O4/c1-15-5-10-27(30-14-15)16(2)24-23(31-27)13-20-18-12-22(29)21-11-17(28)6-8-25(21,3)19(18)7-9-26(20,24)4/h15-21,23-24,28H,5-14H2,1-4H3/t15-,16+,17+,18-,19+,20+,21-,23+,24+,25-,26+,27-/m1/s1"

Replaced current annotation with rescued data. 

----------AERUGINOSIN 98A ----------
Found at least 2 compounds of that name on pubchem.
Found at least 8 compounds with formula C29H45ClN6O9S on pubchem.
No matches found for inchi "InChI=1S/C29H45ClN6O9S/c" 

----- INCHI COULD NOT BE RESCUED FOR SPECTRUM: 7308 -----
----------"5-bromo-N-(4,5-dihydro-1H-imidazol-2-yl)quinoxalin-6-amine" ----------
Found at least 0 compounds of that name on pubchem.
No mat

Found at least 2 compounds of that name on pubchem.
--> FOUND MATCHING COMPOUND ON PUBCHEM.
Inchi ( input ): "InChI=1S/C21H26N2O3/c12621(25)191510172014(13423516(13)2220)8923(17)1112(15)6718(19)24/h25,12,15,1719,22,24H,611H2,1H3"
Inchi (pubchem): "InChI=1S/C21H26N2O3/c1-26-21(25)19-15-10-17-20-14(13-4-2-3-5-16(13)22-20)8-9-23(17)11-12(15)6-7-18(19)24/h2-5,12,15,17-19,22,24H,6-11H2,1H3/t12-,15-,17-,18-,19-/m0/s1"

Replaced current annotation with rescued data. 

----------Corynanthin----------
Found at least 2 compounds of that name on pubchem.
--> FOUND MATCHING COMPOUND ON PUBCHEM.
Inchi ( input ): "InChI=1S/C21H26N2O3/c12621(25)191510172014(13423516(13)2220)8923(17)1112(15)6718(19)24/h25,12,15,1719,22,24H,611H2,1H3"
Inchi (pubchem): "InChI=1S/C21H26N2O3/c1-26-21(25)19-15-10-17-20-14(13-4-2-3-5-16(13)22-20)8-9-23(17)11-12(15)6-7-18(19)24/h2-5,12,15,17-19,22,24H,6-11H2,1H3/t12-,15-,17-,18-,19-/m0/s1"

Replaced current annotation with rescued data. 

----------Corynanthin----------
Foun

Found at least 1 compounds of that name on pubchem.
--> FOUND MATCHING COMPOUND ON PUBCHEM.
Inchi ( input ): "InChI=1S/C25H24O6/c113(2)571521(28)2022(29)17(146818(26)19(27)1114)123024(20)1691025(3,4)3123(15)16/h56,812,2628H,7H2,14H3"
Inchi (pubchem): "InChI=1S/C25H24O6/c1-13(2)5-7-15-21(28)20-22(29)17(14-6-8-18(26)19(27)11-14)12-30-24(20)16-9-10-25(3,4)31-23(15)16/h5-6,8-12,26-28H,7H2,1-4H3"

Replaced current annotation with rescued data. 

----------pomiferin----------
Found at least 1 compounds of that name on pubchem.
--> FOUND MATCHING COMPOUND ON PUBCHEM.
Inchi ( input ): "InChI=1S/C25H24O6/c113(2)571521(28)2022(29)17(146818(26)19(27)1114)123024(20)1691025(3,4)3123(15)16/h56,812,2628H,7H2,14H3"
Inchi (pubchem): "InChI=1S/C25H24O6/c1-13(2)5-7-15-21(28)20-22(29)17(14-6-8-18(26)19(27)11-14)12-30-24(20)16-9-10-25(3,4)31-23(15)16/h5-6,8-12,26-28H,7H2,1-4H3"

Replaced current annotation with rescued data. 

----------pomiferin----------
Found at least 1 compounds of that name on pubchem

...

Found at least 1 compounds of that name on pubchem.
--> FOUND MATCHING COMPOUND ON PUBCHEM.
Inchi ( input ): "InChI=1S/C15H22O5/c114(2)479(614)15(3,19)5810(11(7)16)13(18)2012(8)17/h7,9,11,13,16,1819H,46H2,13H3"
Inchi (pubchem): "InChI=1S/C15H22O5/c1-14(2)4-7-9(6-14)15(3,19)5-8-10(11(7)16)13(18)20-12(8)17/h7,9,11,13,16,18-19H,4-6H2,1-3H3"

Replaced current annotation with rescued data. 

----------1,5,9-trihydroxy-5,7,7-trimethyl-4,5a,6,8,8a,9-hexahydro-1H-azuleno[5,6-c]furan-3-one----------
Found at least 1 compounds of that name on pubchem.
--> FOUND MATCHING COMPOUND ON PUBCHEM.
Inchi ( input ): "InChI=1S/C15H22O5/c114(2)479(614)15(3,19)5810(11(7)16)13(18)2012(8)17/h7,9,11,13,16,1819H,46H2,13H3"
Inchi (pubchem): "InChI=1S/C15H22O5/c1-14(2)4-7-9(6-14)15(3,19)5-8-10(11(7)16)13(18)20-12(8)17/h7,9,11,13,16,18-19H,4-6H2,1-3H3"

Replaced current annotation with rescued data. 

----------candidone----------
Found at least 2 compounds of that name on pubchem.
--> FOUND MATCHING COMPOUND ON P

Found at least 1 compounds of that name on pubchem.
--> FOUND MATCHING COMPOUND ON PUBCHEM.
Inchi ( input ): "InChI=1S/C12H12O5/c1149687(4510(13)178)11(152)12(9)163/h46H,13H3"
Inchi (pubchem): "InChI=1S/C12H12O5/c1-14-9-6-8-7(4-5-10(13)17-8)11(15-2)12(9)16-3/h4-6H,1-3H3"

Replaced current annotation with rescued data. 

----------2',6'-dihydroxy-4'-methoxydihydrochalcone----------
Found at least 1 compounds of that name on pubchem.
--> FOUND MATCHING COMPOUND ON PUBCHEM.
Inchi ( input ): "InChI=1S/C16H16O4/c12012914(18)16(15(19)1012)13(17)87115324611/h26,910,1819H,78H2,1H3"
Inchi (pubchem): "InChI=1S/C16H16O4/c1-20-12-9-14(18)16(15(19)10-12)13(17)8-7-11-5-3-2-4-6-11/h2-6,9-10,18-19H,7-8H2,1H3"

Replaced current annotation with rescued data. 

----------2',6'-dihydroxy-4'-methoxydihydrochalcone----------
Found at least 1 compounds of that name on pubchem.
--> FOUND MATCHING COMPOUND ON PUBCHEM.
Inchi ( input ): "InChI=1S/C16H16O4/c12012914(18)16(15(19)1012)13(17)87115324611/h26,910,1819

Found at least 1 compounds of that name on pubchem.
--> FOUND MATCHING COMPOUND ON PUBCHEM.
Inchi ( input ): "InChI=1S/C12H14O5/c115968(4511(13)14)710(162)12(9)173/h47H,13H3,(H,13,14)/b54+"
Inchi (pubchem): "InChI=1S/C12H14O5/c1-15-9-6-8(4-5-11(13)14)7-10(16-2)12(9)17-3/h4-7H,1-3H3,(H,13,14)/b5-4+"

Replaced current annotation with rescued data. 

----------3,4,5-trimethoxycinnamic acid----------
Found at least 1 compounds of that name on pubchem.
--> FOUND MATCHING COMPOUND ON PUBCHEM.
Inchi ( input ): "InChI=1S/C12H14O5/c115968(4511(13)14)710(162)12(9)173/h47H,13H3,(H,13,14)/b54+"
Inchi (pubchem): "InChI=1S/C12H14O5/c1-15-9-6-8(4-5-11(13)14)7-10(16-2)12(9)17-3/h4-7H,1-3H3,(H,13,14)/b5-4+"

Replaced current annotation with rescued data. 

----------3,4,5-trimethoxycinnamic acid----------
Found at least 1 compounds of that name on pubchem.
--> FOUND MATCHING COMPOUND ON PUBCHEM.
Inchi ( input ): "InChI=1S/C12H14O5/c115968(4511(13)14)710(162)12(9)173/h47H,13H3,(H,13,14)/b54+"
Inchi (pub

Found at least 1 compounds of that name on pubchem.
--> FOUND MATCHING COMPOUND ON PUBCHEM.
Inchi ( input ): "InChI=1S/C20H32O3/c113(1018(22)23)6819(4)14(2)7920(5)15(3)1116(21)1217(19)20/h11,1314,17H,610,12H2,15H3,(H,22,23)"
Inchi (pubchem): "InChI=1S/C20H32O3/c1-13(10-18(22)23)6-8-19(4)14(2)7-9-20(5)15(3)11-16(21)12-17(19)20/h11,13-14,17H,6-10,12H2,1-5H3,(H,22,23)"

Replaced current annotation with rescued data. 

----------5-(1,2,4a,5-tetramethyl-7-oxo-3,4,8,8a-tetrahydro-2H-naphthalen-1-yl)-3-methylpentanoic acid----------
Found at least 1 compounds of that name on pubchem.
--> FOUND MATCHING COMPOUND ON PUBCHEM.
Inchi ( input ): "InChI=1S/C20H32O3/c113(1018(22)23)6819(4)14(2)7920(5)15(3)1116(21)1217(19)20/h11,1314,17H,610,12H2,15H3,(H,22,23)"
Inchi (pubchem): "InChI=1S/C20H32O3/c1-13(10-18(22)23)6-8-19(4)14(2)7-9-20(5)15(3)11-16(21)12-17(19)20/h11,13-14,17H,6-10,12H2,1-5H3,(H,22,23)"

Replaced current annotation with rescued data. 

----------5-(1,2,4a,5-tetramethyl-7-oxo-3,4,8,8a-

Found at least 2 compounds of that name on pubchem.
--> FOUND MATCHING COMPOUND ON PUBCHEM.
Inchi ( input ): "InChI=1S/C27H24O18/c281118(212(29)18(11)34)24(39)4271721(37)23(4425(40)9313(30)19(35)14(31)49)22(38)27(4317)4526(41)10515(32)20(36)16(33)610/h16,17,2123,2738H,7H2/t17,21,22,23+,27+/m1/s1"
Inchi (pubchem): "InChI=1S/C27H24O18/c28-11-1-8(2-12(29)18(11)34)24(39)42-7-17-21(37)23(44-25(40)9-3-13(30)19(35)14(31)4-9)22(38)27(43-17)45-26(41)10-5-15(32)20(36)16(33)6-10/h1-6,17,21-23,27-38H,7H2"

Replaced current annotation with rescued data. 

----------3,9-Dimethoxypterocarpan----------
Found at least 2 compounds of that name on pubchem.
--> FOUND MATCHING COMPOUND ON PUBCHEM.
Inchi ( input ): "InChI=1S/C17H16O4/c11810461315(710)20914125311(192)816(12)2117(13)14/h38,14,17H,9H2,12H3"
Inchi (pubchem): "InChI=1S/C17H16O4/c1-18-10-4-6-13-15(7-10)20-9-14-12-5-3-11(19-2)8-16(12)21-17(13)14/h3-8,14,17H,9H2,1-2H3/t14-,17-/m0/s1"

Replaced current annotation with rescued data. 

----------3,9-D

In [31]:
import numpy as np
en(inchi_corrected), np.sum(np.array(inchi_corrected) == 1)

(496, 488)

### This means that only 8 inchi could not be corrected above!
remaining IDs were: [4027, 4520, 7308, 7932, 90609, 90610, 90611, 146152]

In [29]:
suspected_inchi_issues2 = []

for i, inchi in enumerate([inchi_lst[x[0]] for x in suspected_inchi_issues]):
        
    if inchi not in empty_entry_types:
        if inchi.split('=')[1][0] == '1': # Filter out empty and defective inchi
            # Check if inchi can be handled by openbabel
            smiles = MS_functions.mol_converter(inchi, "inchi", "smi")
            if smiles in empty_entry_types:
                suspected_inchi_issues2.append((i, inchi))
        else:
            suspected_inchi_issues2.append((i, inchi))

In [34]:
[suspected_inchi_issues[x[0]][0] for x in suspected_inchi_issues2]

[4027, 4520, 7308, 7932, 90609, 90610, 90611, 146152]

#### insert save point

In [17]:
"""
# Save data here
import pickle
filename = 'collected_spectra_after_pubchem'

with open(filename, "wb") as output:
    pickle.dump(collected_spectra, output, pickle.HIGHEST_PROTOCOL);
"""

In [3]:
#"""
import pickle
filename = 'collected_spectra_after_pubchem'
collected_spectra = pickle.load(open(filename, "rb"))
#"""

## Check ionization mode (and charge, adducts)

In [4]:
ionmode_lst = []
charge_lst = []
adduct_lst = []

for spectrum in collected_spectra:
    ionmode = 0
    charge = 0
    adduct = None
    for l in spectrum:
        if l.startswith('IONMODE='):
            ionmode = l.split('IONMODE=')[1]
            ionmode = ionmode.replace('\n', '').replace(' ', '').lower()[:8]
            
        if l.startswith('CHARGE='):
            charge = l.split('CHARGE=')[1]
            charge = charge.replace('\n', '').replace(' ', '').lower()
        
        if l.startswith('NAME='):
            adduct = l.split(' ')[-1]
            adduct = adduct.replace('\n', '').replace(' ', '')
            
    ionmode_lst.append(ionmode)
    charge_lst.append(charge)
    adduct_lst.append(adduct)

In [5]:
print(ionmode_lst[:20], charge_lst[:20], adduct_lst[:20])

['negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative'] ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1'] ['M-H', '2M-H', '2M-2H+Na', 'M-H', '2M-H', '2M-2H+Na', 'M-H', '2M-H', '2M-2H+Na', 'M-H', '2M-H', '2M-H', 'M-H', '2M-H', '2M-2H+Na', 'M-H', '2M-H', '2M-H', 'M-H', '2M-H']


In [6]:
ionmode_lst_corrected = [x[:8] for x in ionmode_lst]
[ionmode_lst_corrected.count(x) for x in list(set(ionmode_lst_corrected))]

[11786, 377, 32152, 103332]

In [7]:
list(set(ionmode_lst_corrected))

['', 'n/a', 'negative', 'positive']

In [8]:
adducts_unique = list(set(adduct_lst))
#adducts_unique = [x.replace('[', '').replace(']', '') for x in adducts_unique]
#adducts_unique = list(set(adducts_unique))
adducts_unique_lst = []
for a in adducts_unique:
    if len(a) > 3:
        if a[3] != '.':
            adducts_unique_lst.append(a)
    else:
        adducts_unique_lst.append(a)

print(adducts_unique_lst)

['2M+K', 'iso-C11', '2M-H+Na', 'M-2H2O+H', 'M+OH', '?', 'M+H-C6H10O5', 'M+H-H2O', 'M+-H20+H', 'M-H-C10H20', '[M+CH3COO]-', 'M+H-C9H10O5', '[M-C3H7O2]-', 'M+FA-H', 'M+H-C24H44O-H2O', '[M+15]+', '[M-1]-', '[2M-H]-', 'MCl+2Na', 'M-NH3', 'M-NH3+H', 'M+Na+2K-2H', 'M+H-NO2-C6H4(CH3)OCH2', 'M-HAc', '[M+2H]', 'M-H-C6H9O5SO3H', 'M+H-3H2O', 'M+H-99', 'M+K-2H', 'M+K', 'M+Na-2H', 'M+Na+H2O', 'anteiso-C11', 'M-H', 'M+2H/2', '[M-2H]--', 'M+3H', 'M+H+K', '[M+2H]++', 'M+H-CH4O', 'MBr+H', 'M+H-C4H6', 'M-H-NH3', 'M+H-C5H9NO4', 'M+NH4-H2O', '[M-H2O+H]+', 'M-', 'M]', 'MS+H', 'M2Br81+H', 'M+H-C5H12N2', 'M+2Na-H', 'M+HCl', 'M+H+H2O', 'M+H-C5H14NO4P', 'M+2H', 'M+H-3CH4', '[2M+H]+', 'M-3H2O+H', 'M+H+Na', 'M+MeOH-H', '[2M+Na]+', 'MCl37+2Na', 'M-18', 'M+H-C11H12N2O3', 'anteiso-C13', '2M-H+2Na', 'MBr+Na', 'M3Cl37+Na', 'M+H-C2H6O', 'M+H-NH3', 'M++', 'M+Cl', '[M]+', '3M+NH4', 'M-H-C3H5NO2', 'M+ACN+H', 'M+Na', 'M+?', 'M-H2O', 'MBr81+Na', 'M-H-H2O', '[M+Li]*+', 'M-H2O-H', '[M+Na]+', '[M+K]', '[M-H2O]', '4M-H', '[M+H

In [9]:
frequent_adducts = []
for adduct in adducts_unique_lst:
    if adduct_lst.count(adduct) > 10:
        frequent_adducts.append(adduct)
print(frequent_adducts)

['M-2H2O+H', 'M+H-H2O', '[M+CH3COO]-', 'M+FA-H', '[M+2H]', 'M+K', 'M-H', 'M+3H', '[M+2H]++', '[M-H2O+H]+', 'M-', 'M+2H', 'M+H+Na', 'M+H-NH3', 'M+Cl', '[M]+', 'M+ACN+H', 'M+Na', 'M-H-H2O', 'M-H2O-H', '[M+Na]+', '[(M+CH3COOH)-H]-', '[M-H]-', 'M+2H]', '[M+Na]', '[M]-', '[M-2H2O+H]+', '[M+CH3COO]-/[M-CH3]-', 'M', 'M+H', '2M-2H+Na', 'M+H-CH3NH2', 'Cat', 'M+NH4', '[M+H]', '[M+K]+', '[M+H]+', 'M+H-2H2O', '[M-H]-/[M-Ser]-', '2M-H', '[M-2H]-', '--', '[M+]', '2M+Na', '[M]+*', '[M]*+', 'Unk', 'M-H2O+H', 'M+', '[M+NH4]+', '2M+H']


### Manually pick adducts where we were certain what ionmode they meant

In [10]:
known_adduct_positive = ['M-2H2O+H', 'M+H', 'M+H-CH3NH2', '[M+Na]', '[M-H2O+H]+','M+H+Na',
                        '[M+H]+', 'M+K', '[M+H]', '[M+K]+', '2M+Na', '[M]*+', 'M+3H', '[M+2H]++',
                        'M+NH4', 'M+ACN+H', 'M+H-NH3', 'M-H2O+H', 'M+', '[M+Na]+', '[M+]', 'M+2H]',
                        'M+H-H2O', '[M-2H2O+H]+', '[M]+', '[M+NH4]+', 'Cat', '[M]+*', 'M+Na',
                        '2M+H', 'M+H-2H2O', '[M+2H]', 'M+2H']
known_adduct_negative = ['[M+CH3COO]-/[M-CH3]-', 'M-H', '[M+CH3COO]-', '[M]-', '2M-H', 
                        '[M-H]-/[M-Ser]-', 'M-', '[M-H]-', '[M-2H]-', 'M-H2O-H', 'M+FA-H',
                        'M+Cl', '[(M+CH3COOH)-H]-', 'M-H-H2O', '[M-H-CO2-2HF]-']

print([x.replace(' ', '').replace('[', '').replace(']','').replace('*', '') for x in known_adduct_negative])
print([x.replace(' ', '').replace('[', '').replace(']','').replace('*', '') for x in known_adduct_positive])

['M+CH3COO-/M-CH3-', 'M-H', 'M+CH3COO-', 'M-', '2M-H', 'M-H-/M-Ser-', 'M-', 'M-H-', 'M-2H-', 'M-H2O-H', 'M+FA-H', 'M+Cl', '(M+CH3COOH)-H-', 'M-H-H2O', 'M-H-CO2-2HF-']
['M-2H2O+H', 'M+H', 'M+H-CH3NH2', 'M+Na', 'M-H2O+H+', 'M+H+Na', 'M+H+', 'M+K', 'M+H', 'M+K+', '2M+Na', 'M+', 'M+3H', 'M+2H++', 'M+NH4', 'M+ACN+H', 'M+H-NH3', 'M-H2O+H', 'M+', 'M+Na+', 'M+', 'M+2H', 'M+H-H2O', 'M-2H2O+H+', 'M+', 'M+NH4+', 'Cat', 'M+', 'M+Na', '2M+H', 'M+H-2H2O', 'M+2H', 'M+2H']


### Check charges in MGF

In [11]:
list(set(charge_lst))

['0', '4+', '1+', '2', '0+', '-1+', '2+', '10+', '1', '3', '3+']

In [12]:
[charge_lst.count(x) for x in list(set(charge_lst))]

[488, 1, 3693, 201, 791, 213, 239, 1, 141976, 27, 17]

## Add missing ionmode information (and charge sign) if possible

In [13]:
def get_ionmode(adduct):
    known_adducts_positive = ['M-2H2O+H', 'M+H', 'M+H-CH3NH2', 'M+Na', 'M-H2O+H+', 'M+H+Na', 
                             'M+H+', 'M+K', 'M+H', 'M+K+', '2M+Na', 'M+', 'M+3H', 'M+2H++', 
                             'M+NH4', 'M+ACN+H', 'M+H-NH3', 'M-H2O+H', 'M+', 'M+Na+', 'M+', 
                             'M+2H', 'M+H-H2O', 'M-2H2O+H+', 'M+', 'M+NH4+', 'Cat', 'M+', 
                             'M+Na', '2M+H', 'M+H-2H2O', 'M+2H', 'M+2H']
    known_adducts_negative = ['M+CH3COO-/M-CH3-', 'M-H', 'M+CH3COO-', 'M-', '2M-H', 'M-H-/M-Ser-', 
                             'M-', 'M-H-', 'M-2H-', 'M-H2O-H', 'M+FA-H', 'M+Cl', '(M+CH3COOH)-H-', 
                             'M-H-H2O', 'M-H-CO2-2HF-']
    
    adduct = adduct.replace(' ', '').replace('[', '').replace(']','').replace('*', '')
    if adduct in known_adducts_positive:
        return 'positive'
    elif adduct in known_adducts_negative:
        return 'negative'
    else:
        return 'n/a'
    

In [14]:
get_ionmode('M-2H2O+H')

'positive'

In [15]:
# clean up charge values...
for i, spectrum in enumerate(collected_spectra):
    ionmode = 'n/a'
    charge = 0
    adduct = None
    
    ionmode_line = None
    charge_line = None
    name_line = 1
    for m, l in enumerate(spectrum):      
        if l.startswith('IONMODE='):
            ionmode = l.split('IONMODE=')[1]
            ionmode = ionmode.replace('\n', '').replace(' ', '').lower()[:8]
            ionmode_line = m
            
        if l.startswith('CHARGE='):
            charge = l.split('CHARGE=')[1]
            charge = charge.replace('\n', '').replace(' ', '').lower()
            charge_line = m
        
        if l.startswith('NAME='):
            adduct = l.split(' ')[-1]
            adduct = adduct.replace('\n', '').replace(' ', '')
            name_line = m

    # Check ionmode and charge sign:
    if charge == '-1+':
        charge = '1-'

    # Just checking where things are inconsistent
    if charge.startswith('-') or charge.endswith('-'):
        if ionmode == 'positive':
            print("Mode and charge sign don't match! Changed sign.") 
            print(ionmode, charge, adduct)
            
        #elif ionmode != 'negative':
        #    ionmode = 'negative'

    if charge.startswith('+') or charge.endswith('+'):
        if ionmode == 'negative':
            print("Mode and charge sign don't match! Changed sign.")
            print(ionmode, charge, adduct)
        #elif ionmode != 'positive':
        #    ionmode = 'positive'
        
    if ionmode not in ['positive', 'negative']:
        ionmode = get_ionmode(adduct)
        print("Added ionmode=", ionmode, "based on adduct:", adduct, ".")

    # Charge field doesn't have a sign
    if ionmode == 'positive':
        charge = charge.replace('-', '').replace('+', '') + '+'
    if ionmode == 'negative':
        charge = charge.replace('-', '').replace('+', '') + '-'
    
    # Add changes to spectrum
    if ionmode_line is not None:
        spectrum[ionmode_line] = 'IONMODE=' + ionmode + '\n'
    else:
        print("no ionmode field found for spectrum", i)
        spectrum.insert(name_line+1, 'IONMODE=' + ionmode + '\n')
    if charge_line is not None:
        spectrum[charge_line] = 'CHARGE=' + charge + '\n'
    else:
        print("no charge field found for spectrum", i)
        spectrum.insert(name_line+1, 'CHARGE=' + charge + '\n')

Added ionmode= positive based on adduct: M+H .
Added ionmode= positive based on adduct: M+2H .
Added ionmode= positive based on adduct: M+2H .
Added ionmode= positive based on adduct: M+H .
Added ionmode= positive based on adduct: M+2H .
Added ionmode= positive based on adduct: M+H .
Added ionmode= positive based on adduct: M+2H .
Added ionmode= positive based on adduct: M+H .
Added ionmode= positive based on adduct: M+2H .
Added ionmode= positive based on adduct: M+2H .
Added ionmode= positive based on adduct: M+2H .
Added ionmode= positive based on adduct: M+2H .
Added ionmode= positive based on adduct: M+H .
Added ionmode= positive based on adduct: M+2H .
Added ionmode= positive based on adduct: M+2H .
Added ionmode= positive based on adduct: M+2H .
Added ionmode= positive based on adduct: M+2H .
Added ionmode= positive based on adduct: M+2H .
Added ionmode= positive based on adduct: M+2H .
Added ionmode= positive based on adduct: M+2H .
Added ionmode= positive based on adduct: M+H 

Added ionmode= positive based on adduct: M+H .
Added ionmode= positive based on adduct: M+H .
Added ionmode= positive based on adduct: M+H .
Added ionmode= positive based on adduct: M+H .
Added ionmode= positive based on adduct: M+H .
Added ionmode= positive based on adduct: M+H .
Added ionmode= positive based on adduct: M+2H .
Added ionmode= positive based on adduct: M+H .
Added ionmode= positive based on adduct: M+H .
Added ionmode= positive based on adduct: M+2H .
Added ionmode= positive based on adduct: M+H .
Added ionmode= positive based on adduct: M+H .
Added ionmode= positive based on adduct: M+H .
Added ionmode= positive based on adduct: M+H .
Added ionmode= positive based on adduct: M+H .
Added ionmode= positive based on adduct: M+H .
Added ionmode= positive based on adduct: M+2H .
Added ionmode= positive based on adduct: M+2H .
Added ionmode= positive based on adduct: M+H .
Added ionmode= positive based on adduct: M+2H .
Added ionmode= positive based on adduct: M+2H .
Added i

Mode and charge sign don't match! Changed sign.
negative 2+ M-2H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 2+ M-2H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M+Na-2H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 0+ M-H
Mode and charge sign don't match! Changed sign.
negative 0+ M-H
Mode and charge sign don't match! Changed sign.
negative 0+ M-H
Mode and charge sign don't match! Changed sign.
negative 0+ M-H
Mode and charge sign don't match! Changed sign.
positive 1- M-H
Mode and charge sign don't match! Changed sign.
positive 1- M-H
Mode and charge sign don't match! 

Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Change

Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 0+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Changed sign.
negative 1+ M-H
Mode and charge sign don't match! Change

...

Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .


Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= n/a based on adduct: [M-2H]-- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= n/a based on adduct: [M-2H]-- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added 

Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .


Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= positive based on adduct: [M+H]+ .
Added ionmode= positive based on adduct: [M+H]+ .
Added ionmode= positive based on adduct: [M+H]+ .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= positive based on adduct: [M+H]+ .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= positive based on adduct: [M+H]+ .
Added ionmode= positive based on adduct: [M+H]+ .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= positive based on adduct: [M+H]+ .
Added ionmode= positive based on adduct: [M+H]+ .
Added ionmode= positive based on adduct: [M+H]+ .
Added ionmode= positive based on adduct: [M+H]+ .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .


Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .
Added ionmode= negative based on adduct: [M-H]- .


In [16]:
ionmode_lst = []
charge_lst = []
adduct_lst = []

for spectrum in collected_spectra:
    ionmode = 0
    charge = 0
    adduct = None
    for l in spectrum:
        if l.startswith('IONMODE='):
            ionmode = l.split('IONMODE=')[1]
            ionmode = ionmode.replace('\n', '').replace(' ', '').lower()[:8]
            
        if l.startswith('CHARGE='):
            charge = l.split('CHARGE=')[1]
            charge = charge.replace('\n', '').replace(' ', '').lower()
        
        if l.startswith('NAME='):
            adduct = l.split(' ')[-1]
            adduct = adduct.replace('\n', '').replace(' ', '')
            
    ionmode_lst.append(ionmode)
    charge_lst.append(charge)
    adduct_lst.append(adduct)

In [17]:
#Numbers before cleaning:
print([32152, 11786, 103332, 377]) #['negative', '', 'positive', 'n/a']

[32152, 11786, 103332, 377]


In [18]:
#Numbers before cleaning:
ionmode_lst_corrected = [x[:8] for x in ionmode_lst]
[ionmode_lst_corrected.count(x) for x in list(set(ionmode_lst_corrected))]

[18, 40339, 107290]

In [19]:
list(set(ionmode_lst_corrected))

['n/a', 'negative', 'positive']

In [20]:
collected_spectra[0]

['BEGIN IONS\n',
 'PEPMASS=391.285\n',
 'CHARGE=1-\n',
 'MSLEVEL=2\n',
 'SOURCE_INSTRUMENT=ESI-Orbitrap\n',
 'FILENAME=Standards_n_1ugmL_chenodeoxycholic.mzXML\n',
 'SEQ=*..*\n',
 'IONMODE=negative\n',
 'ORGANISM=BILELIB19\n',
 'NAME=chenodeoxycholic acid M-H\n',
 'PI=Dorrestein\n',
 'DATACOLLECTOR=A. Jarmusch\n',
 'SMILES=C[C@H](CCC(=O)O)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3[C@H]2[C@@H](C[C@H]4[C@@]3(CC[C@H](C4)O)C)O)C\n',
 'INCHI="InChI=1S/C24H40O4/c1-14(4-7-21(27)28)17-5-6-18-22-19(9-11-24(17,18)3)23(2)10-8-16(25)12-15(23)13-20(22)26/h14-20,22,25-26H,4-13H2,1-3H3,(H,27,28)/t14?,15-,16+,17+,18-,19-,20+,22-,23?,24?/m0/s1"\n',
 'INCHIKEY=RUDATBOHQWOJDD-GYHJSIKNSA-N\n',
 'PUBMED=N/A\n',
 'SUBMITUSER=mpanitchpakdi\n',
 'TAGS=\n',
 'LIBRARYQUALITY=1\n',
 'SPECTRUMID=CCMSLIB00005435444\n',
 'SCANS=1688\n',
 '51.787151\t18114.681641\n',
 '58.22208\t18008.097656\n',
 '71.187103\t21309.207031\n',
 '75.990234\t20840.488281\n',
 '80.070091\t20326.140625\n',
 '91.880928\t23875.916016\n',
 '95.083244\t2

### Make empty smiles, inchi, inchikeys more homogeneous

In [21]:
empty_entry_types = ['N/A', 'n/a', 'NA', 0, '0', '""', '', 
                     'nodata', '"InChI=n/a"', '"InChI="', 
                     'InChI=1S/N\n', '\t\r\n']

"""
# An empirically found list of strings that occured in the dataset and represent empty entries

"""

inchi_lst = []
inchikey_lst = []
smiles_lst = []
inchiaux_lst = []

for i, spectrum in enumerate(collected_spectra):
    # Show progress
    if i // 10000 == i/10000:
        print("running spectra cleaning no", i)
        
    smiles = 'N/A'
    inchi = 'N/A'
    inchikey = 'N/A'
    inchiaux = 'N/A'
    
    spectrum_new = []
    for l in spectrum:
        if l.startswith('SMILES='):
            smiles = l.strip().split('SMILES=')[1].replace(" ", "")
            if smiles in empty_entry_types:
                smiles = 'N/A'
                l = 'SMILES=N/A\n'
                
        if l.startswith('INCHI='):
            inchi = l.strip().split('INCHI=')[1].replace(" ", "")
            if inchi in empty_entry_types:
                inchi = 'N/A'
                l = 'INCHI=N/A\n'
                
        if l.startswith('INCHIKEY='):
            inchikey = l.strip().split('INCHIKEY=')[1].replace(" ", "")
            if inchikey in empty_entry_types:
                inchikey = 'N/A'
                l = 'INCHIKEY=N/A\n'
                
        if l.startswith('INCHIAUX='):
            inchiaux = l.strip().split('INCHIAUX=')[1].replace(" ", "")
        
        if l != '\n': # Skip empty lines
            spectrum_new.append(l)

    # Add to lists
    smiles_lst.append(smiles)
    inchi_lst.append(inchi)
    inchikey_lst.append(inchikey)
    inchiaux_lst.append(inchiaux)
    
    # Add cleaned spectrum back to collection
    collected_spectra[i] = spectrum_new  

running spectra cleaning no 0
running spectra cleaning no 10000
running spectra cleaning no 20000
running spectra cleaning no 30000
running spectra cleaning no 40000
running spectra cleaning no 50000
running spectra cleaning no 60000
running spectra cleaning no 70000
running spectra cleaning no 80000
running spectra cleaning no 90000
running spectra cleaning no 100000
running spectra cleaning no 110000
running spectra cleaning no 120000
running spectra cleaning no 130000
running spectra cleaning no 140000


In [19]:
# Just checking if we didn't miss some more inchikeys (that were wrongly put into inchiaux line)
num_inchiaux = len([x for x in inchiaux_lst if x not in empty_entry_types])
print("Number of spectra with inchiaux:", num_inchiaux)

Number of spectra with inchiaux: 0


### Inspect some sample spectra

In [20]:
inchikey_lst[:10]

['RUDATBOHQWOJDD-GYHJSIKNSA-N',
 'RUDATBOHQWOJDD-PODLAYBOSA-N',
 'RUDATBOHQWOJDD-PODLAYBOSA-N',
 'BHQCQFFYRZLCQQ-IHELEQLESA-N',
 'BHQCQFFYRZLCQQ-IHELEQLESA-N',
 'BHQCQFFYRZLCQQ-IHELEQLESA-N',
 'KXGVEGMKQFWNSR-BKAWJTANSA-N',
 'KXGVEGMKQFWNSR-BKAWJTANSA-N',
 'KXGVEGMKQFWNSR-BKAWJTANSA-N',
 'GHCZAUBVMUEKKP-AJEXLSFKSA-N']

In [21]:
inchi_lst[:5], smiles_lst[:5]

(['"InChI=1S/C24H40O4/c1-14(4-7-21(27)28)17-5-6-18-22-19(9-11-24(17,18)3)23(2)10-8-16(25)12-15(23)13-20(22)26/h14-20,22,25-26H,4-13H2,1-3H3,(H,27,28)/t14?,15-,16+,17+,18-,19-,20+,22-,23?,24?/m0/s1"',
  '"InChI=1S/C24H40O4/c1-14(4-7-21(27)28)17-5-6-18-22-19(9-11-24(17,18)3)23(2)10-8-16(25)12-15(23)13-20(22)26/h14-20,22,25-26H,4-13H2,1-3H3,(H,27,28)/t14?,15-,16+,17+,18-,19-,20+,22-,23?,24?/m0/s2"',
  '"InChI=1S/C24H40O4/c1-14(4-7-21(27)28)17-5-6-18-22-19(9-11-24(17,18)3)23(2)10-8-16(25)12-15(23)13-20(22)26/h14-20,22,25-26H,4-13H2,1-3H3,(H,27,28)/t14?,15-,16+,17+,18-,19-,20+,22-,23?,24?/m0/s2"',
  '"InChI=1S/C24H40O5/c1-13(4-7-21(28)29)16-5-6-17-22-18(12-20(27)24(16,17)3)23(2)9-8-15(25)10-14(23)11-19(22)26/h13-20,22,25-27H,4-12H2,1-3H3,(H,28,29)/t13?,14-,15+,16+,17-,18-,19+,20-,22-,23?,24?/m0/s1"',
  '"InChI=1S/C24H40O5/c1-13(4-7-21(28)29)16-5-6-17-22-18(12-20(27)24(16,17)3)23(2)9-8-15(25)10-14(23)11-19(22)26/h13-20,22,25-27H,4-12H2,1-3H3,(H,28,29)/t13?,14-,15+,16+,17-,18-,19+,20-,22-,23?

### Check number of annotations found (again)

In [22]:
num_inchi = len([x for x in inchi_lst if x not in empty_entry_types])
print("Number of spectra with inchi:", num_inchi)

num_inchikey = len([x for x in inchikey_lst if x not in empty_entry_types])
print("Number of spectra with inchikey:", num_inchikey)

num_smiles = len([x for x in smiles_lst if x not in empty_entry_types])
print("Number of spectra with smiles:", num_smiles)

Number of spectra with inchi: 87852
Number of spectra with inchikey: 87850
Number of spectra with smiles: 87848


## Inspect the very few remaining cases where we still don't have inchi AND smiles

In [23]:
inchi_no_smiles = []
for i, smiles in enumerate(smiles_lst):
    if smiles in empty_entry_types:
        if inchi_lst[i] not in empty_entry_types:
            inchi_no_smiles.append(i)
            
len(inchi_no_smiles)

4

In [24]:
[inchi_lst[ID] + " smiles: " + smiles_lst[ID] + " inchikey: " + inchikey_lst[ID] for ID in inchi_no_smiles]

['"InChI=Brc1c(NC2=NCCN2)ccc3nccnc13" smiles: N/A inchikey: N/A',
 '"InChI=1S/C16H14O3/c11812781513(912)14(17)1016(1915)115324611/h29,16H,10H2,1H3" smiles: N/A inchikey: GMPRRYBAFNHXFW-UHFFFAOYSA-N',
 '"InChI=1S/C16H14O3/c11812781513(912)14(17)1016(1915)115324611/h29,16H,10H2,1H3" smiles: N/A inchikey: GMPRRYBAFNHXFW-UHFFFAOYSA-N',
 '"InChI=1S/C16H14O3/c11812781513(912)14(17)1016(1915)115324611/h29,16H,10H2,1H3" smiles: N/A inchikey: GMPRRYBAFNHXFW-UHFFFAOYSA-N']

In [25]:
smiles_no_inchi = []
for i, inchi in enumerate(inchi_lst):
    if inchi in empty_entry_types:
        if smiles_lst[i] not in empty_entry_types:
            smiles_no_inchi.append(i)
            
len(smiles_no_inchi)

0

### Correct final defect in strings

In [32]:
# remove wrongly placed line breaks
for i, spectrum in enumerate(collected_spectra):
    correct = False
    for m, l in enumerate(spectrum):
        if l.endswith('\n"\n'):
            l = l.replace('\n"\n', '"\n')
            spectrum[m] = l
            correct = True
        if l.endswith('\t\r\n'):
            l = l.replace('\t\r\n', '\n')
            spectrum[m] = l
            correct = True           
    if correct:
        collected_spectra[i] = spectrum

In [27]:
# add missing line breaks
for i, spectrum in enumerate(collected_spectra):
    correct = False
    for m, l in enumerate(spectrum):
        if not l.endswith('\n'):
            l = l + '\n'
            spectrum[m] = l
            correct = True
    if correct:
        collected_spectra[i] = spectrum

In [33]:
collected_spectra[3724]

['BEGIN IONS\n',
 'TITLE=2O-rhamnosyl-swertisin\n',
 'LIBRARYQUALITY=3\n',
 'SMILES=C[C@H]1[C@@H]([C@H]([C@H]([C@@H](O1)O[C@@H]1[C@H]([C@@H]([C@@H](CO)O[C@H]1c1c(cc2c(c(=O)cc(c3ccc(cc3)O)o2)c1O)OC)O)O)O)O)O\n',
 'INCHIKEY=UINZSMIFSIQGBZ-KVQGGKNTSA-N\n',
 'INCHI="InChI=1S/C28H32O14/c1-10-20(32)23(35)25(37)28(39-10)42-27-24(36)21(33)17(9-29)41-26(27)19-15(38-2)8-16-18(22(19)34)13(31)7-14(40-16)11-3-5-12(30)6-4-11/h3-8,10,17,20-21,23-30,32-37H,9H2,1-2H3/t10-,17+,20-,21+,23+,24-,25+,26-,27+,28-/m0/s1"\n',
 'PEPMASS=591.16901\n',
 'CHARGE=1-\n',
 'MSLEVEL=2\n',
 'SOURCE_INSTRUMENT=DI-ESI-qTof\n',
 'FILENAME=f.rsilva/gnps_2_O_rhamnosyl_swertisin_neg_MSMS_591.mgf;\n',
 'SEQ=*..*\n',
 'NOTES=\n',
 'IONMODE=negative\n',
 'ORGANISM=GNPS-LIBRARY\n',
 'NAME=2O-rhamnosyl-swertisin M-H\n',
 'SPECTRUMID=CCMSLIB00000077222\n',
 'ACTIVATION=CID\n',
 'INSTRUMENT=ion trap\n',
 'SCANS=1\n',
 '231.029861 123.000000\n',
 '253.047745 144.000000\n',
 '281.045044 401.000000\n',
 '282.049194 187.000000\n',
 '28

## Write collected_spectra list into MGF file:

In [34]:
file_mgf = os.path.join(PATH_MS_DATA, 'all_gnps_extensively_corrected_191106.mgf')
with open(file_mgf, 'w') as f:
    for c in collected_spectra:
        for l in c:
            try:
                f.write(l.encode('utf8', 'replace').decode('utf8', 'replace'))
            except UnicodeEncodeError:
                f.write(l.encode('ascii', 'replace').decode('utf8', 'replace'))
                print(l)
        f.write('\n')

TITLE=C16 Galactosyl(�) Ceramide (d18:1/16:0)

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=11,22-dimethyl (6E,17E)-3,14-dioxo-8,19-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})-4,9,15,20-tetraoxatricyclo[16.4.0.0?,��]docosa-6,10,17,21-tetraene-11,22-dicarboxylate

TITLE=11,22-dimethyl (6E,17E)-3,14-dioxo-8

TITLE=5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5,14-dicarboxylic acid

TITLE=5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5,14-dicarboxylic acid

TITLE=14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carbaldehyde

TITLE=14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carbaldehyde

TITLE=(1S,4S,5R,10S,13S,17S,19S,20R)-10-hydroxy-4,5,9,9,13,19,20-heptamethyl-24-oxahexacyclo[15.5.2.0�,�?.0?,�?.0?,�?.0?,��]tetracos-15-en-23-one

TITLE=(1S,4S,5R,10S,13S,17S,19S,20R)-10-hydroxy-4,5,9,9,13,19,20-heptamethyl-24-oxahexacyclo[15.5.2.0�,�?.0?,�?.0?,�?.0?,��]tetracos-15-en-23-one

TITLE=(1S,4S,5R,10S,13S,17S,19S,20R)-10-hydroxy-4,5,9,9,13,19,20-heptamethyl-24-oxahexacyclo[15.5.2.0�,�?.0?,�?.0?,�?.0?,��]tetracos-15-en-23-one

TITLE=(1S,4S,5R,10S,13S,17S,19S,20R)-10-hydroxy-4,5,9,9,13,19,20-heptamethyl-24-oxahexacyclo[15.5.2.0�,�?.0?,�?.0?,�?.0?,��]tetracos-15-en-23-one

TITLE=(1R,3R,6R,8S,10R,13S,16S,17R)-8-tert-butyl-6,17-dihydrox

TITLE=methyl (2R)-2-[(1S,3S,7R,8R,9R,12S,13R)-13-(furan-3-yl)-6,6,8,12-tetramethyl-17-methylidene-5,15-dioxo-2,14-dioxatetracyclo[7.7.1.0�,��.0�,?]heptadecan-7-yl]-2-hydroxyacetate

TITLE=methyl (2R)-2-[(1S,3S,7R,8R,9R,12S,13R)-13-(furan-3-yl)-6,6,8,12-tetramethyl-17-methylidene-5,15-dioxo-2,14-dioxatetracyclo[7.7.1.0�,��.0�,?]heptadecan-7-yl]-2-hydroxyacetate

TITLE=methyl (2R)-2-[(1S,3S,7R,8R,9R,12S,13R)-13-(furan-3-yl)-6,6,8,12-tetramethyl-17-methylidene-5,15-dioxo-2,14-dioxatetracyclo[7.7.1.0�,��.0�,?]heptadecan-7-yl]-2-hydroxyacetate

TITLE=methyl (2R)-2-(acetyloxy)-2-[(1S,3S,7R,8R,9R,12S,13R)-13-(furan-3-yl)-6,6,8,12-tetramethyl-17-methylidene-5,15-dioxo-2,14-dioxatetracyclo[7.7.1.0�,��.0�,?]heptadecan-7-yl]acetate

TITLE=methyl (2R)-2-(acetyloxy)-2-[(1S,3S,7R,8R,9R,12S,13R)-13-(furan-3-yl)-6,6,8,12-tetramethyl-17-methylidene-5,15-dioxo-2,14-dioxatetracyclo[7.7.1.0�,��.0�,?]heptadecan-7-yl]acetate

TITLE=[(1S,2R,4S,5R,9R,10S,13R,17S,20R,22S)?2?[(3,4?dihydroxy?5?{[3,4,5?trihydroxy

TITLE=(2S,3S,9R,12R,13R,14S,15R,16R,17S)-3,12,13,15,16-pentahydroxy-2,6,14,17-tetramethyl-10-oxatetracyclo[7.7.1.0�,?.0��,�?]heptadec-5-ene-4,11-dione

TITLE=(2S,3S,9R,12R,13R,14S,15R,16R,17S)-3,12,13,15,16-pentahydroxy-2,6,14,17-tetramethyl-10-oxatetracyclo[7.7.1.0�,?.0��,�?]heptadec-5-ene-4,11-dione

TITLE=(2S,3S,9R,12R,13R,14S,15R,16R,17S)-3,12,13,15,16-pentahydroxy-2,6,14,17-tetramethyl-10-oxatetracyclo[7.7.1.0�,?.0��,�?]heptadec-5-ene-4,11-dione

TITLE=(2S,3S,9R,12R,13R,14S,15R,16R,17S)-3,12,13,15,16-pentahydroxy-2,6,14,17-tetramethyl-10-oxatetracyclo[7.7.1.0�,?.0��,�?]heptadec-5-ene-4,11-dione

TITLE=(1'S,3R,3'R,5R,6'R,7'S)-5-(furan-3-yl)-3',6',7'-trimethyl-9'-oxaspiro[oxolane-3,2'-tricyclo[5.2.2.0�,?]undecane]-2,8'-dione

TITLE=(2R,3R,6R,8R,9S,12S,13R,14R,15R,16R)-6,8,14,15-tetrahydroxy-2,6,13,16-tetramethyl-3-{[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-10-oxatetracyclo[7.6.1.0�,?.0��,�?]hexadec-4-en-11-one

TITLE=(2R,3R,6R,8R,9S,12S,13R,14R,15R,16R)-6,8,

TITLE=11,22-dimethyl (6E,17E)-3,14-dioxo-8,19-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})-4,9,15,20-tetraoxatricyclo[16.4.0.0?,��]docosa-6,10,17,21-tetraene-11,22-dicarboxylate

TITLE=11,22-dimethyl (6E,17E)-3,14-dioxo-8,19-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})-4,9,15,20-tetraoxatricyclo[16.4.0.0?,��]docosa-6,10,17,21-tetraene-11,22-dicarboxylate

TITLE=11,22-dimethyl (6E,17E)-3,14-dioxo-8,19-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})-4,9,15,20-tetraoxatricyclo[16.4.0.0?,��]docosa-6,10,17,21-tetraene-11,22-dicarboxylate

TITLE=11,22-dimethyl (6E,17E)-3,14-dioxo-8,19-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})-4,9,15,20-tetraoxatricyclo[16.4.0.0?,��]docosa-6,10,17,21-tetraene-11,22-dicarboxylate

TITLE=11,22-dimethyl (6E,17E)-3,14-dioxo-8,19-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})-4,9,15,20-tetraoxatricyclo[16.4.0.0?,��]docosa-6,10,1

...

TITLE=(2E)-6-{13,17-dihydroxy-7,7,12,16-tetramethyl-6-[(3,4,5-trihydroxyoxan-2-yl)oxy]pentacyclo[9.7.0.0�,�.0�,?.0��,�?]octadecan-15-yl}-2-methylhept-2-enoic acid

TITLE=(2E)-6-{13,17-dihydroxy-7,7,12,16-tetramethyl-6-[(3,4,5-trihydroxyoxan-2-yl)oxy]pentacyclo[9.7.0.0�,�.0�,?.0��,�?]octadecan-15-yl}-2-methylhept-2-enoic acid

TITLE=(2E)-6-{13,17-dihydroxy-7,7,12,16-tetramethyl-6-[(3,4,5-trihydroxyoxan-2-yl)oxy]pentacyclo[9.7.0.0�,�.0�,?.0��,�?]octadecan-15-yl}-2-methylhept-2-enoic acid

TITLE=(2E)-6-{13,17-dihydroxy-7,7,12,16-tetramethyl-6-[(3,4,5-trihydroxyoxan-2-yl)oxy]pentacyclo[9.7.0.0�,�.0�,?.0��,�?]octadecan-15-yl}-2-methylhept-2-enoic acid

TITLE=(2E)-6-{13,17-dihydroxy-7,7,12,16-tetramethyl-6-[(3,4,5-trihydroxyoxan-2-yl)oxy]pentacyclo[9.7.0.0�,�.0�,?.0��,�?]octadecan-15-yl}-2-methylhept-2-enoic acid

TITLE=(2E)-6-{13,17-dihydroxy-7,7,12,16-tetramethyl-6-[(3,4,5-trihydroxyoxan-2-yl)oxy]pentacyclo[9.7.0.0�,�.0�,?.0��,�?]octadecan-15-yl}-2-methylhept-2-enoic acid

TITLE=(2E)-6-{13

TITLE=(2R,7S,13R,14R,16S,19R,20S)-19-(furan-3-yl)-9,9,13,20-tetramethyl-5,12,17-trioxo-4,8,15,18-tetraoxahexacyclo[11.9.0.0�,?.0�,�?.0�?,�?.0�?,�?]docosan-11-yl acetate

TITLE=(2R,7S,13R,14R,16S,19R,20S)-19-(furan-3-yl)-9,9,13,20-tetramethyl-5,12,17-trioxo-4,8,15,18-tetraoxahexacyclo[11.9.0.0�,?.0�,�?.0�?,�?.0�?,�?]docosan-11-yl acetate

TITLE=(2R,7S,13R,14R,16S,19R,20S)-19-(furan-3-yl)-9,9,13,20-tetramethyl-5,12,17-trioxo-4,8,15,18-tetraoxahexacyclo[11.9.0.0�,?.0�,�?.0�?,�?.0�?,�?]docosan-11-yl acetate

TITLE=(2R,7S,13R,14R,16S,19R,20S)-19-(furan-3-yl)-9,9,13,20-tetramethyl-5,12,17-trioxo-4,8,15,18-tetraoxahexacyclo[11.9.0.0�,?.0�,�?.0�?,�?.0�?,�?]docosan-11-yl acetate

TITLE=(2R,7S,13R,14R,16S,19R,20S)-19-(furan-3-yl)-9,9,13,20-tetramethyl-5,12,17-trioxo-4,8,15,18-tetraoxahexacyclo[11.9.0.0�,?.0�,�?.0�?,�?.0�?,�?]docosan-11-yl acetate

TITLE=(2R,7S,13R,14R,16S,19R,20S)-19-(furan-3-yl)-9,9,13,20-tetramethyl-5,12,17-trioxo-4,8,15,18-tetraoxahexacyclo[11.9.0.0�,?.0�,�?.0�?,�?.0�?,�?]doc

TITLE=(1R,2S,4R,5S,6S,11R,12R,14S,16R)-5,6,11-trihydroxy-4-(hydroxymethyl)-8,12,15,15-tetramethyl-7-oxo-3-oxapentacyclo[9.5.0.0�,?.0?,�?.0�?,�?]hexadec-8-en-14-yl dodecanoate

TITLE=(1R,2S,4R,5S,6S,11R,12R,14S,16R)-5,6,11-trihydroxy-4-(hydroxymethyl)-8,12,15,15-tetramethyl-7-oxo-3-oxapentacyclo[9.5.0.0�,?.0?,�?.0�?,�?]hexadec-8-en-14-yl dodecanoate

TITLE=(1R,2S,4R,5S,6S,11R,12R,14S,16R)-5,6,11-trihydroxy-4-(hydroxymethyl)-8,12,15,15-tetramethyl-7-oxo-3-oxapentacyclo[9.5.0.0�,?.0?,�?.0�?,�?]hexadec-8-en-14-yl dodecanoate

TITLE=(1R,2S,4R,5S,6S,11R,12R,14S,16R)-5,6,11-trihydroxy-4-(hydroxymethyl)-8,12,15,15-tetramethyl-7-oxo-3-oxapentacyclo[9.5.0.0�,?.0?,�?.0�?,�?]hexadec-8-en-14-yl dodecanoate

TITLE=(1R,2S,4R,5S,6S,11R,12R,14S,16R)-5,6,11-trihydroxy-4-(hydroxymethyl)-8,12,15,15-tetramethyl-7-oxo-3-oxapentacyclo[9.5.0.0�,?.0?,�?.0�?,�?]hexadec-8-en-14-yl dodecanoate

TITLE=(1R,2S,4R,5S,6S,11R,12R,14S,16R)-5,6,11-trihydroxy-4-(hydroxymethyl)-8,12,15,15-tetramethyl-7-oxo-3-oxapentacyclo[


TITLE=4,5-dihydroxy-4,5,6-trimethyl-2,8-dioxa-13-azatricyclo[8.5.1.0��,�?]hexadec-10-ene-3,7-dione

TITLE=4,5-dihydroxy-4,5,6-trimethyl-2,8-dioxa-13-azatricyclo[8.5.1.0��,�?]hexadec-10-ene-3,7-dione

TITLE=4,5-dihydroxy-4,5,6-trimethyl-2,8-dioxa-13-azatricyclo[8.5.1.0��,�?]hexadec-10-ene-3,7-dione

TITLE=4,5-dihydroxy-4,5,6-trimethyl-2,8-dioxa-13-azatricyclo[8.5.1.0��,�?]hexadec-10-ene-3,7-dione

TITLE=4,5-dihydroxy-4,5,6-trimethyl-2,8-dioxa-13-azatricyclo[8.5.1.0��,�?]hexadec-10-ene-3,7-dione

TITLE=(1S,2S,3R,4S,7R,10S,12R,15S)-4,12-bis(acetyloxy)-1,9,15-trihydroxy-10,14,17,17-tetramethyl-11-oxo-6-oxatetracyclo[11.3.1.0�,�?.0?,?]heptadec-13-en-2-yl benzoate

TITLE=(1S,2S,3R,4S,7R,10S,12R,15S)-4,12-bis(acetyloxy)-1,9,15-trihydroxy-10,14,17,17-tetramethyl-11-oxo-6-oxatetracyclo[11.3.1.0�,�?.0?,?]heptadec-13-en-2-yl benzoate

TITLE=(1S,2S,3R,4S,7R,10S,12R,15S)-4,12-bis(acetyloxy)-1,9,15-trihydroxy-10,14,17,17-tetramethyl-11-oxo-6-oxatetracyclo[11.3.1.0�,�?.0?,?]heptadec-13-en-2-yl benzo

TITLE=(5S,9R)-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=(5S,9R)-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=(5S,9R)-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=(15S,16Z,17S)-16-ethylidene-4-hydroxy-15-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-2,10,14,20-tetraoxatetracyclo[21.2.2.1�,?.0��,�?]octacosa-1(25),3(28),4,6,12,23,26-heptaene-11,19-dione

TITLE=(15S,16Z,17S)-16-ethylidene-4-hydroxy-15-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-2,10,14,20-tetraoxatetracyclo[21.2.2.1�,?.0��,�?]octacosa-1(25),3(28),4,6,12,23,26-heptaene-11,19-dione

TITLE=(15S,16Z,17S)-16-ethylidene-4-hydroxy-15-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-2,10,14,20-tetraoxatetracyclo[21.2.2.1�,?.0��,�?]octacosa-1(25),3(28),4,6,12,23,26-heptaene-11,19-dione

TITLE=(15S,16Z,17S)-16-ethyl

## Replace charge values of "-1+" with "-1"
Re-write mgf file line by line and replace "-1+" by "-1". Not elegant. But works.

In [62]:
"""input_file = os.path.join(PATH_MS_DATA, 'all_gnps_extensively_corrected_191106.mgf')
output_file = open(os.path.join(ROOT, ",Data", "GNPS_all", "MS_data_allGNPS_191106_corrected.mgf"), 'w')

with open(input_file, 'r') as file:
    for line in file:
        output_file.write(line.replace('-1+', '-1'))
"""

#### correct defective line discovered during importing

In [36]:
input_file = os.path.join(PATH_MS_DATA, 'all_gnps_extensively_corrected_191106.mgf')
output_file = open(os.path.join(ROOT, ",Data", "GNPS_all", "MS_data_allGNPS_191106_corrected.mgf"), 'w'

with open(input_file, 'r') as file:
    for line in file:
        output_file.write(line.replace('PUBMEND IONS', 'END IONS'))

In [63]:
output_file.close()

---
# Split dataset into positive and negative ionization mode

In [37]:
ionmode_lst.count('negative'), ionmode_lst.count('positive')

(40339, 107290)

In [38]:
# get IDs of all spectra with positive/negative ionization mode
IDs_positive = [x for x in range(len(collected_spectra)) if ionmode_lst[x] == 'positive']
IDs_negative = [x for x in range(len(collected_spectra)) if ionmode_lst[x] == 'negative']
len(IDs_negative), len(IDs_positive)

(40339, 107290)

In [39]:
file_mgf = os.path.join(PATH_MS_DATA, 'allGNPS_positive_191106.mgf')
with open(file_mgf, 'w') as f:
    for ID in IDs_positive:
        spec = collected_spectra[ID]
        for l in spec:
            try:
                f.write(l.encode('utf8', 'replace').decode('utf8', 'replace'))
            except UnicodeEncodeError:
                f.write(l.encode('ascii', 'replace').decode('utf8', 'replace'))
                print(l)
        f.write('\n')

TITLE=C16 Galactosyl(�) Ceramide (d18:1/16:0)

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=11,22-dimethyl (6E,17E)-3,14-dioxo-8,19-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})-4,9,15,20-tetraoxatricyclo[16.4.0.0?,��]docosa-6,10,17,21-tetraene-11,22-dicarboxylate

TITLE=11,22-dimethyl (6E,17E)-3,14-dioxo-8,19-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})-4,9,15,20-tetraoxatricyclo[16.4.0.0?,��]docosa-6,10,17,21-tetraene-11,22-dicarboxylate

TITLE=(1R,2E,8S,10R,11S)-10,11-dihydroxy-6-(methoxymethyl)-1,10-dimethyl-5-oxo-4,14-dioxatricyclo[9.2.1.0�,?]tetradeca-2,6-dien-8-yl 2-methylprop-2-enoate

TITLE=(1R,2E,8S,10R,11S)-10,11-

TITLE=2,4-dihydroxy-5-methoxy-1'',6'',8''-trimethyldispiro[bis(oxolane)-3,2':5',7''-[3]oxatricyclo[6.3.1.0?,��]dodecane]-2''-one

TITLE=(1S,3R,6R,7S,8S,10R,13S,16S,17R)-8-tert-butyl-6,17-dihydroxy-16-methyl-2,4,14,19-tetraoxahexacyclo[8.7.2.0�,��.0�,?.0?,��.0��,�?]nonadecane-5,15,18-trione

TITLE=(1S,3R,6R,7S,8S,10R,13S,16S,17R)-8-tert-butyl-6,17-dihydroxy-16-methyl-2,4,14,19-tetraoxahexacyclo[8.7.2.0�,��.0�,?.0?,��.0��,�?]nonadecane-5,15,18-trione

TITLE=(5S,9R)-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=methyl (2R)-2-[(1S,2R,4S,9R,10R,14S,15S,17S)-17-(acetyloxy)-9-(furan-3-yl)-1-hydroxy-10,14,16,16-tetramethyl-7,18-dioxo-3,8-dioxapentacyclo[12.3.1.0�,?.0?,��.0?,�?]octadecan-15-yl]-2-hydroxyacetate

TITLE=methyl (2R)-2-[(1S,2R,4S,9R,10R,14S,15S,17S)-17-(acetyloxy)-9-(furan-3-yl)-1-hydroxy-10,14,16,16-tetramethyl-7,18-dioxo-3,8-dioxapentacyclo[12.3.1.0�,?.0?,��.0?,�?]octadecan-15-yl]-2-hydroxyacetate

TITLE=(1S,2R,4S,9R,10R,14S,15S,1

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0

TITLE=10-[(6-carboxy-4-{[4,5-dihydroxy-6-(hydroxymethyl)-3-[(3,4,5-trihydroxy-6-methyloxan-2-yl)oxy]oxan-2-yl]oxy}-5-hydroxy-3-{[3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}oxan-2-yl)oxy]-2-hydroxy-4,5,9,9,13,20-hexamethyl-24-oxahexacyclo[15.5.2.0�,�?.0?,�?.0?,�?.0?,��]tetracosane-20-carboxylic acid

TITLE=10-[(6-carboxy-4-{[4,5-dihydroxy-6-(hydroxymethyl)-3-[(3,4,5-trihydroxy-6-methyloxan-2-yl)oxy]oxan-2-yl]oxy}-5-hydroxy-3-{[3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}oxan-2-yl)oxy]-2-hydroxy-4,5,9,9,13,20-hexamethyl-24-oxahexacyclo[15.5.2.0�,�?.0?,�?.0?,�?.0?,��]tetracosane-20-carboxylic acid

TITLE=10-[(6-carboxy-4-{[4,5-dihydroxy-6-(hydroxymethyl)-3-[(3,4,5-trihydroxy-6-methyloxan-2-yl)oxy]oxan-2-yl]oxy}-5-hydroxy-3-{[3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}oxan-2-yl)oxy]-2-hydroxy-4,5,9,9,13,20-hexamethyl-24-oxahexacyclo[15.5.2.0�,�?.0?,�?.0?,�?.0?,��]tetracosane-20-carboxylic acid

TITLE=10-[(6-carboxy-4-{[4,5-dihydroxy-6-(hydroxymethyl)-3-[(3,4,5-trihydroxy-6-me

TITLE=14-hydroxy-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=14-hydroxy-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=14-hydroxy-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=14-hydroxy-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=14-hydroxy-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=14-hydroxy-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=14-hydroxy-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=14-hydroxy-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=14-hydroxy-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=14-hydroxy-14-(hydroxymethyl)-5

...


TITLE=13-methoxy-1,6-diazatetracyclo[7.6.1.0?,�?.0�?,�?]hexadeca-3,5,7,9(16),10,12,14-heptaen-2-one

TITLE=13-methoxy-1,6-diazatetracyclo[7.6.1.0?,�?.0�?,�?]hexadeca-3,5,7,9(16),10,12,14-heptaen-2-one

TITLE=13-methoxy-1,6-diazatetracyclo[7.6.1.0?,�?.0�?,�?]hexadeca-3,5,7,9(16),10,12,14-heptaen-2-one

TITLE=13-methoxy-1,6-diazatetracyclo[7.6.1.0?,�?.0�?,�?]hexadeca-3,5,7,9(16),10,12,14-heptaen-2-one

TITLE=13-methoxy-1,6-diazatetracyclo[7.6.1.0?,�?.0�?,�?]hexadeca-3,5,7,9(16),10,12,14-heptaen-2-one

TITLE=13-methoxy-1,6-diazatetracyclo[7.6.1.0?,�?.0�?,�?]hexadeca-3,5,7,9(16),10,12,14-heptaen-2-one

TITLE=4-[(7R,11S,14S,16R)-14-[(4-methoxy-6-methyl-5-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-({[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}methyl)oxan-2-yl]oxy}oxan-2-yl)oxy]-7,11-dimethyl-2-oxapentacyclo[8.8.0.0�,�.0�,?.0��,�?]octadec-5-en-6-yl]-2,5-dihydrofuran-2-one

TITLE=4-[(7R,11S,14S,16R)-14-[(4-methoxy-6-methyl-5-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-({[(2R,3R,4S,


TITLE=14-(acetyloxy)-1,6-dihydroxy-8-(hydroxymethyl)-4,12,12,15-tetramethyl-5-oxotetracyclo[8.5.0.0�,?.0��,��]pentadeca-3,8-dien-13-yl 2-methylbutanoate

TITLE=14-(acetyloxy)-1,6-dihydroxy-8-(hydroxymethyl)-4,12,12,15-tetramethyl-5-oxotetracyclo[8.5.0.0�,?.0��,��]pentadeca-3,8-dien-13-yl 2-methylbutanoate

TITLE=14-(acetyloxy)-1,6-dihydroxy-8-(hydroxymethyl)-4,12,12,15-tetramethyl-5-oxotetracyclo[8.5.0.0�,?.0��,��]pentadeca-3,8-dien-13-yl 2-methylbutanoate

TITLE=14-(acetyloxy)-1,6-dihydroxy-8-(hydroxymethyl)-4,12,12,15-tetramethyl-5-oxotetracyclo[8.5.0.0�,?.0��,��]pentadeca-3,8-dien-13-yl 2-methylbutanoate

TITLE=14-(acetyloxy)-1,6-dihydroxy-8-(hydroxymethyl)-4,12,12,15-tetramethyl-5-oxotetracyclo[8.5.0.0�,?.0��,��]pentadeca-3,8-dien-13-yl 2-methylbutanoate

TITLE=14-(acetyloxy)-1,6-dihydroxy-8-(hydroxymethyl)-4,12,12,15-tetramethyl-5-oxotetracyclo[8.5.0.0�,?.0��,��]pentadeca-3,8-dien-13-yl 2-methylbutanoate

TITLE=14-(acetyloxy)-1,6-dihydroxy-8-(hydroxymethyl)-4,12,12,15-tetramethyl

TITLE=(2S,3R,4R,5R,6S)-2-[(4-hydroxy-6-{[(4S,6R,7S,8R,9S,13S,16S)-6-hydroxy-7,9,13-trimethyl-6-[(3R)-3-methyl-4-{[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}butyl]-5-oxapentacyclo[10.8.0.0�,?.0?,?.0��,�?]icosan-16-yl]oxy}-2-(hydroxymethyl)-5-{[3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}oxan-3-yl)oxy]-6-methyloxane-3,4,5-triol

TITLE=(2S,3R,4R,5R,6S)-2-[(4-hydroxy-6-{[(4S,6R,7S,8R,9S,13S,16S)-6-hydroxy-7,9,13-trimethyl-6-[(3R)-3-methyl-4-{[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}butyl]-5-oxapentacyclo[10.8.0.0�,?.0?,?.0��,�?]icosan-16-yl]oxy}-2-(hydroxymethyl)-5-{[3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}oxan-3-yl)oxy]-6-methyloxane-3,4,5-triol

TITLE=(2S,3R,4R,5R,6S)-2-[(4-hydroxy-6-{[(4S,6R,7S,8R,9S,13S,16S)-6-hydroxy-7,9,13-trimethyl-6-[(3R)-3-methyl-4-{[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}butyl]-5-oxapentacyclo[10.8.0.0�,?.0?,?.0��,�?]icosan-16-yl]oxy}-2-(hydroxymethyl)-5-{[3,4,5-trihydroxy-6-(hy

TITLE=methyl 8,11,15,20,22-pentahydroxy-24-methyl-13,18,27-trioxo-6-oxaheptacyclo[15.10.2.0�,�?.0�,�?.0?,�?.0?,��.0��,�?]nonacosa-3,5(14),11,15,19,21(26),22,24,28-nonaene-7-carboxylate

TITLE=methyl 8,11,15,20,22-pentahydroxy-24-methyl-13,18,27-trioxo-6-oxaheptacyclo[15.10.2.0�,�?.0�,�?.0?,�?.0?,��.0��,�?]nonacosa-3,5(14),11,15,19,21(26),22,24,28-nonaene-7-carboxylate

TITLE=methyl 8,11,15,20,22-pentahydroxy-24-methyl-13,18,27-trioxo-6-oxaheptacyclo[15.10.2.0�,�?.0�,�?.0?,�?.0?,��.0��,�?]nonacosa-3,5(14),11,15,19,21(26),22,24,28-nonaene-7-carboxylate

TITLE=methyl 8,11,15,20,22-pentahydroxy-24-methyl-13,18,27-trioxo-6-oxaheptacyclo[15.10.2.0�,�?.0�,�?.0?,�?.0?,��.0��,�?]nonacosa-3,5(14),11,15,19,21(26),22,24,28-nonaene-7-carboxylate

TITLE=methyl 8,11,15,20,22-pentahydroxy-24-methyl-13,18,27-trioxo-6-oxaheptacyclo[15.10.2.0�,�?.0�,�?.0?,�?.0?,��.0��,�?]nonacosa-3,5(14),11,15,19,21(26),22,24,28-nonaene-7-carboxylate

TITLE=methyl 8,11,15,20,22-pentahydroxy-24-methyl-13,18,27-trioxo-6-ox

TITLE=13-hydroxy-7,14-dimethoxy-6-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-2,9-dioxatetracyclo[6.6.2.0?,�?.0��,�?]hexadeca-1(15),4(16),5,7,11,13-hexaene-3,10-dione

TITLE=13-hydroxy-7,14-dimethoxy-6-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-2,9-dioxatetracyclo[6.6.2.0?,�?.0��,�?]hexadeca-1(15),4(16),5,7,11,13-hexaene-3,10-dione

TITLE=13-hydroxy-7,14-dimethoxy-6-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-2,9-dioxatetracyclo[6.6.2.0?,�?.0��,�?]hexadeca-1(15),4(16),5,7,11,13-hexaene-3,10-dione

TITLE=13-hydroxy-7,14-dimethoxy-6-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-2,9-dioxatetracyclo[6.6.2.0?,�?.0��,�?]hexadeca-1(15),4(16),5,7,11,13-hexaene-3,10-dione

TITLE=13-hydroxy-7,14-dimethoxy-6-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-2,9-dioxatetracyclo[6.6.2.0?,�?.0��,�?]hexadeca-1(15),4(16),5,7,11,13-hexaene-3,10-dione

TITLE=13-hydroxy-7,14-dimethoxy-6-{[(2S,

In [40]:
file_mgf = os.path.join(PATH_MS_DATA, 'allGNPS_negative_191106.mgf')
with open(file_mgf, 'w') as f:
    for ID in IDs_negative:
        spec = collected_spectra[ID]
        for l in spec:
            try:
                f.write(l.encode('utf8', 'replace').decode('utf8', 'replace'))
            except UnicodeEncodeError:
                f.write(l.encode('ascii', 'replace').decode('utf8', 'replace'))
                print(l)
        f.write('\n')

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=11,22-dimethyl (6E,17E)-3,14-dioxo-8,19-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})-4,9,15,20-tetraoxatricyclo[16.4.0.0?,��]docosa-6,10,17,21-tetraene-11,22-dicarboxylate

TITLE=11,22-dimethyl (6E,17E)-3,14-dioxo-8,19-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})-4,9,15,20-tetraoxatricyclo[16.4.0.0?,��]docosa-6,10,17,21-tetraene-11,22-dicarboxylate

TITLE=2,4,18-trihydroxy-15-(3-hydroxy-3-methylbutyl)-7,9,13-trioxapentacyclo[10.8.0.0�,�?.0?,?.0�?,�?]icosa-1(12),2,5,10,14,16,18-heptaen-20-one

TITLE=2,4,18-trihydroxy-15-(3-hydroxy-3-methylbutyl)-7,9,13-trioxapentacyclo[10.8.0.0�,�

TITLE=7-hydroxy-2,6,9,12,17,17-hexamethyl-22-methylidene-5,16,19,21-tetraoxaheptacyclo[10.8.1.1�,?.0�,��.0�,?.0��,�?.0�?,�?]docos-13-ene-4,8,15-trione

TITLE=7-hydroxy-2,6,9,12,17,17-hexamethyl-22-methylidene-5,16,19,21-tetraoxaheptacyclo[10.8.1.1�,?.0�,��.0�,?.0��,�?.0�?,�?]docos-13-ene-4,8,15-trione

TITLE=(2S,3R,4R,5R,6S)-2-{[(2R,3S,4S,5R,6R)-5-{[(2S,3R,4S,5R,6R)-3,5-dihydroxy-6-(hydroxymethyl)-4-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}oxan-2-yl]oxy}-4-hydroxy-2-(hydroxymethyl)-6-[(1'S,2S,4'S,5S,7'S,8'R,9'S,13'R,16'S)-5,7',9',13'-tetramethyl-5-({[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}methyl)-5'-oxaspiro[oxolane-2,6'-pentacyclo[10.8.0.0�,?.0?,?.0��,�?]icosan]-18'-eneoxy]oxan-3-yl]oxy}-6-methyloxane-3,4,5-triol

TITLE=(2S,3R,4R,5R,6S)-2-{[(2R,3S,4S,5R,6R)-5-{[(2S,3R,4S,5R,6R)-3,5-dihydroxy-6-(hydroxymethyl)-4-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}oxan-2-yl]oxy}-4-hydroxy-2-(hydroxymethyl)-6-[(1'S,2S,4'S

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=2-[(5S,25R,26R,29S,30S,31S)-13,14,15,18,19,20,31,35,36-nonahydroxy-2,10,23,28,32-pentaoxo-5-(3,4,5-trihydroxybenzoyloxy)-3,6,9,24,27,33-hexaoxaheptacyclo[28.7.1.0?,�?.0?,�?.0��,�?.0�?,��.0�?,�?]octatriaconta-1(38),11(16),12,14,17,19,21,34,36-nonaen-29-yl]acetic acid

TITLE=2-[(5S,25R,26R,29S,30S,31S)-13,14,15,18,19,20,31,35,36-nonahydroxy-2,10,23,28,32-pentaoxo-5-(3,4,5-trihydroxybenzoyloxy)-3,6,9,24,27,33-hexaoxaheptacyclo[28.7.1.0?,�?.0?,�?.0��,�?.0�?,��.0�?,�?]octatriaconta-1(38),11(16),12,14,17,19,21,34,36-nonaen-29-yl]acetic acid

TITLE=2-[(5S,25R,26R,29S,30S,31S)-13,14,15,18,19,20,31,35,36-nonahydroxy-2,10,23,28,32-pentaoxo-5-(3,4,5-trihydroxybenzoyloxy)-3,6,9,24,27,3

TITLE=15'-[(5-{[3,4-dihydroxy-6-(hydroxymethyl)-5-{[3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}oxan-2-yl]oxy}-3-hydroxy-4-methoxy-6-methyloxan-2-yl)oxy]-7'-hydroxy-8',12'-dimethyl-6'-oxaspiro[oxolane-3,5'-pentacyclo[9.8.0.0�,?.0?,?.0��,�?]nonadecane]-5-one

TITLE=15'-[(5-{[3,4-dihydroxy-6-(hydroxymethyl)-5-{[3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}oxan-2-yl]oxy}-3-hydroxy-4-methoxy-6-methyloxan-2-yl)oxy]-7'-hydroxy-8',12'-dimethyl-6'-oxaspiro[oxolane-3,5'-pentacyclo[9.8.0.0�,?.0?,?.0��,�?]nonadecane]-5-one

TITLE=15'-[(5-{[3,4-dihydroxy-6-(hydroxymethyl)-5-{[3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}oxan-2-yl]oxy}-3-hydroxy-4-methoxy-6-methyloxan-2-yl)oxy]-7'-hydroxy-8',12'-dimethyl-6'-oxaspiro[oxolane-3,5'-pentacyclo[9.8.0.0�,?.0?,?.0��,�?]nonadecane]-5-one

TITLE=(2R,7S,13R,14R,16S,19R,20S)-19-(furan-3-yl)-11-hydroxy-9,9,13,20-tetramethyl-4,8,15,18-tetraoxahexacyclo[11.9.0.0�,?.0�,�?.0�?,�?.0�?,�?]docos-10-ene-5,12,17-trione

TITLE=(2R,7S,13R,14R,16S,19R,20S)-19-(f

TITLE=(1S,8S,9S,10S,13R)-6,9,10-trimethyl-2-oxo-4,14-dioxatetracyclo[7.5.0.0�,��.0�,?]tetradeca-3(7),5-dien-8-yl acetate

TITLE=(1S,8S,9S,10S,13R)-6,9,10-trimethyl-2-oxo-4,14-dioxatetracyclo[7.5.0.0�,��.0�,?]tetradeca-3(7),5-dien-8-yl acetate

TITLE=(1S,8S,9S,10S,13R)-6,9,10-trimethyl-2-oxo-4,14-dioxatetracyclo[7.5.0.0�,��.0�,?]tetradeca-3(7),5-dien-8-yl acetate

TITLE=(4E)-8-hydroxy-4-(1-hydroxypropan-2-ylidene)-10-oxatricyclo[7.2.1.0�,?]dodecane-8-carboxylic acid

TITLE=(4E)-8-hydroxy-4-(1-hydroxypropan-2-ylidene)-10-oxatricyclo[7.2.1.0�,?]dodecane-8-carboxylic acid

TITLE=(4E)-8-hydroxy-4-(1-hydroxypropan-2-ylidene)-10-oxatricyclo[7.2.1.0�,?]dodecane-8-carboxylic acid

TITLE=(2R,3S,4S,9R,12R,13R,14S,15R,16R,17S)-3,4,12,13,15,16-hexahydroxy-2,6,14,17-tetramethyl-10-oxatetracyclo[7.7.1.0�,?.0��,�?]heptadec-6-en-11-one

TITLE=(2R,3S,4S,9R,12R,13R,14S,15R,16R,17S)-3,4,12,13,15,16-hexahydroxy-2,6,14,17-tetramethyl-10-oxatetracyclo[7.7.1.0�,?.0��,�?]heptadec-6-en-11-one

TITLE=(2R,3S,4S,9

TITLE=methyl (1S,12S,13E)-13-ethylidene-18-(hydroxymethyl)-4-methoxy-8,15-diazapentacyclo[10.5.1.0�,?.0�,?.0�?,�?]octadeca-2,4,6,8-tetraene-18-carboxylate

TITLE=methyl (1S,12S,13E)-13-ethylidene-18-(hydroxymethyl)-4-methoxy-8,15-diazapentacyclo[10.5.1.0�,?.0�,?.0�?,�?]octadeca-2,4,6,8-tetraene-18-carboxylate

TITLE=methyl (1S,12S,13E)-13-ethylidene-18-(hydroxymethyl)-4-methoxy-8,15-diazapentacyclo[10.5.1.0�,?.0�,?.0�?,�?]octadeca-2,4,6,8-tetraene-18-carboxylate

TITLE=6-(furan-3-yl)-13-hydroxy-5,12,16,16-tetramethyl-7,10,17-trioxahexacyclo[13.3.3.0�,�?.0�,��.0?,��.0?,��]henicos-19-ene-8,14,21-trione

TITLE=6-(furan-3-yl)-13-hydroxy-5,12,16,16-tetramethyl-7,10,17-trioxahexacyclo[13.3.3.0�,�?.0�,��.0?,��.0?,��]henicos-19-ene-8,14,21-trione

TITLE=6-(furan-3-yl)-13-hydroxy-5,12,16,16-tetramethyl-7,10,17-trioxahexacyclo[13.3.3.0�,�?.0�,��.0?,��.0?,��]henicos-19-ene-8,14,21-trione

TITLE=methyl (1S,4S,8R,11S,14S)-11-ethyl-12-oxo-7,9,13-trioxatetracyclo[6.5.1.0�,�?.0?,�?]tetradeca-2,5-diene

TITLE=9-methoxy-4,4-dimethyl-13-phenyl-6,14-dioxatetracyclo[8.4.0.0�,?.0�,?]tetradeca-1,7,9-trien-11-one

TITLE=9-methoxy-4,4-dimethyl-13-phenyl-6,14-dioxatetracyclo[8.4.0.0�,?.0�,?]tetradeca-1,7,9-trien-11-one

TITLE=9-methoxy-4,4-dimethyl-13-phenyl-6,14-dioxatetracyclo[8.4.0.0�,?.0�,?]tetradeca-1,7,9-trien-11-one

TITLE=11-(hydroxymethyl)-2,6,6,14,19-pentamethyl-7,16,18-trioxapentacyclo[12.6.1.0�,��.0?,�?.0�?,��]henicosa-5(10),11-diene-3,8,15,20-tetrone

TITLE=11-(hydroxymethyl)-2,6,6,14,19-pentamethyl-7,16,18-trioxapentacyclo[12.6.1.0�,��.0?,�?.0�?,��]henicosa-5(10),11-diene-3,8,15,20-tetrone

TITLE=11-(hydroxymethyl)-2,6,6,14,19-pentamethyl-7,16,18-trioxapentacyclo[12.6.1.0�,��.0?,�?.0�?,��]henicosa-5(10),11-diene-3,8,15,20-tetrone

TITLE=(3S,5R,8S)-2-[(benzoyloxy)methyl]-8-methyl-3-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-9,10-dioxatetracyclo[4.3.1.0�,?.0�,?]decane-6-sulfonic acid

TITLE=(3S,5R,8S)-2-[(benzoyloxy)methyl]-8-methyl-3-{[(2S,3R,4S,5S,6R)-3,4

TITLE=(2R,3R,6R,8R,9S,12S,13R,14R,15R,16R)-6,8,14,15-tetrahydroxy-2,6,13,16-tetramethyl-3-{[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-10-oxatetracyclo[7.6.1.0�,?.0��,�?]hexadec-4-en-11-one

TITLE=(2R,3R,6R,8R,9S,12S,13R,14R,15R,16R)-6,8,14,15-tetrahydroxy-2,6,13,16-tetramethyl-3-{[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-10-oxatetracyclo[7.6.1.0�,?.0��,�?]hexadec-4-en-11-one

TITLE=(2R,3R,6R,8R,9S,12S,13R,14R,15R,16R)-6,8,14,15-tetrahydroxy-2,6,13,16-tetramethyl-3-{[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-10-oxatetracyclo[7.6.1.0�,?.0��,�?]hexadec-4-en-11-one

TITLE=13-methoxy-1,6-diazatetracyclo[7.6.1.0?,�?.0�?,�?]hexadeca-3,5,7,9(16),10,12,14-heptaen-2-one

TITLE=13-methoxy-1,6-diazatetracyclo[7.6.1.0?,�?.0�?,�?]hexadeca-3,5,7,9(16),10,12,14-heptaen-2-one

TITLE=13-methoxy-1,6-diazatetracyclo[7.6.1.0?,�?.0�?,�?]hexadeca-3,5,7,9(16),10,12,14-heptaen-2-one

TITLE=4-[(7R,11S,14S,16R)-14-[(4-methoxy-6-methyl-5-{[(

TITLE=N-[(3S,6S,9S,11R,15S,18S,20R,21R,24S,25S,26S)-3-[(1R)-2-carbamoyl-1-hydroxyethyl]-6-[(1S,2S)-1,2-dihydroxy-2-(4-hydroxyphenyl)ethyl]-11,20,21,25-tetrahydroxy-15-[(1R)-1-hydroxyethyl]-26-methyl-2,5,8,14,17,23-hexaoxo-1,4,7,13,16,22-hexaazatricyclo[22.3.0.0?,��]heptacosan-18-yl]-10,12-dimethyltetradecanamide

TITLE=N-[(3S,6S,9S,11R,15S,18S,20R,21R,24S,25S,26S)-3-[(1R)-2-carbamoyl-1-hydroxyethyl]-6-[(1S,2S)-1,2-dihydroxy-2-(4-hydroxyphenyl)ethyl]-11,20,21,25-tetrahydroxy-15-[(1R)-1-hydroxyethyl]-26-methyl-2,5,8,14,17,23-hexaoxo-1,4,7,13,16,22-hexaazatricyclo[22.3.0.0?,��]heptacosan-18-yl]-10,12-dimethyltetradecanamide

TITLE=(2S,3R,4R,5R,6S)-2-{[(2R,3S,4R,5R,6S)-6-{[(2R,3R,4S,5S,6R)-4,5-dihydroxy-6-(hydroxymethyl)-2-[(1'S,2S,4'S,5S,7'R,9'S,13'R,16'S)-5,7',9',13'-tetramethyl-5-({[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}methyl)-5'-oxaspiro[oxolane-2,6'-pentacyclo[10.8.0.0�,?.0?,?.0��,�?]icosan]-18'-eneoxy]oxan-3-yl]oxy}-4,5-dihydroxy-2-(hydroxymethyl)oxan-3-yl

---
## Create subsets of only spectra with full annotations

In [41]:
# get IDs of all spectra with proper smiles
IDs_positive_smiles = [x for x in IDs_positive if smiles_lst[x] not in empty_entry_types]
IDs_negative_smiles = [x for x in IDs_negative if smiles_lst[x] not in empty_entry_types]

In [42]:
len(IDs_negative_smiles), len(IDs_positive_smiles)

(23255, 64575)

In [43]:
file_mgf = os.path.join(PATH_MS_DATA, 'allGNPS_positive_smiles_191106.mgf')
with open(file_mgf, 'w') as f:
    for ID in IDs_positive_smiles:
        spec = collected_spectra[ID]
        for l in spec:
            try:
                f.write(l.encode('utf8', 'replace').decode('utf8', 'replace'))
            except UnicodeEncodeError:
                f.write(l.encode('ascii', 'replace').decode('utf8', 'replace'))
                print(l)
        f.write('\n')

TITLE=C16 Galactosyl(�) Ceramide (d18:1/16:0)

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=11,22-dimethyl (6E,17E)-3,14-dioxo-8,19-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})-4,9,15,20-tetraoxatricyclo[16.4.0.0?,��]docosa-6,10,17,21-tetraene-11,22-dicarboxylate

TITLE=11,22-dimethyl (6E,17E)-3,14-dioxo-8,19-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})-4,9,15,20-tetraoxatricyclo[16.4.0.0?,��]docosa-6,10,17,21-tetraene-11,22-dicarboxylate

TITLE=(1R,2E,8S,10R,11S)-10,11-dihydroxy-6-(methoxymethyl)-1,10-dimethyl-5-oxo-4,14-dioxatricyclo[9.2.1.0�,?]tetradeca-2,6-dien-8-yl 2-methylprop-2-enoate

TITLE=(1R,2E,8S,10R,11S)-10,11-

TITLE=4,5-dihydroxy-4,5,6-trimethyl-2,8-dioxa-13-azatricyclo[8.5.1.0��,�?]hexadec-10-ene-3,7-dione

TITLE=4,5-dihydroxy-4,5,6-trimethyl-2,8-dioxa-13-azatricyclo[8.5.1.0��,�?]hexadec-10-ene-3,7-dione

TITLE=(1S,2S,3R,4S,7R,10S,12R,15S)-4,12-bis(acetyloxy)-1,9,15-trihydroxy-10,14,17,17-tetramethyl-11-oxo-6-oxatetracyclo[11.3.1.0�,�?.0?,?]heptadec-13-en-2-yl benzoate

TITLE=methyl (1R,18R)-17-ethyl-3,13-diazapentacyclo[13.3.1.0�,�?.0?,?.0��,�?]nonadeca-2(10),4(9),5,7,16-pentaene-1-carboxylate

TITLE=methyl 8,11,15,20,22-pentahydroxy-24-methyl-13,18,27-trioxo-6-oxaheptacyclo[15.10.2.0�,�?.0�,�?.0?,�?.0?,��.0��,�?]nonacosa-3,5(14),11,15,19,21(26),22,24,28-nonaene-7-carboxylate

TITLE=methyl 8,11,15,20,22-pentahydroxy-24-methyl-13,18,27-trioxo-6-oxaheptacyclo[15.10.2.0�,�?.0�,�?.0?,�?.0?,��.0��,�?]nonacosa-3,5(14),11,15,19,21(26),22,24,28-nonaene-7-carboxylate

TITLE=N-[(3S,6S,9S,11R,15S,18S,20R,21R,24S,25S,26S)-3-[(1R)-2-carbamoyl-1-hydroxyethyl]-6-[(1S,2S)-1,2-dihydroxy-2-(4-hydroxyphenyl)

TITLE=10-[(6-carboxy-4-{[4,5-dihydroxy-6-(hydroxymethyl)-3-[(3,4,5-trihydroxy-6-methyloxan-2-yl)oxy]oxan-2-yl]oxy}-5-hydroxy-3-{[3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}oxan-2-yl)oxy]-2-hydroxy-4,5,9,9,13,20-hexamethyl-24-oxahexacyclo[15.5.2.0�,�?.0?,�?.0?,�?.0?,��]tetracosane-20-carboxylic acid

TITLE=10-[(6-carboxy-4-{[4,5-dihydroxy-6-(hydroxymethyl)-3-[(3,4,5-trihydroxy-6-methyloxan-2-yl)oxy]oxan-2-yl]oxy}-5-hydroxy-3-{[3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}oxan-2-yl)oxy]-2-hydroxy-4,5,9,9,13,20-hexamethyl-24-oxahexacyclo[15.5.2.0�,�?.0?,�?.0?,�?.0?,��]tetracosane-20-carboxylic acid

TITLE=10-[(6-carboxy-4-{[4,5-dihydroxy-6-(hydroxymethyl)-3-[(3,4,5-trihydroxy-6-methyloxan-2-yl)oxy]oxan-2-yl]oxy}-5-hydroxy-3-{[3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}oxan-2-yl)oxy]-2-hydroxy-4,5,9,9,13,20-hexamethyl-24-oxahexacyclo[15.5.2.0�,�?.0?,�?.0?,�?.0?,��]tetracosane-20-carboxylic acid

TITLE=10-[(6-carboxy-4-{[4,5-dihydroxy-6-(hydroxymethyl)-3-[(3,4,5-trihydroxy-6-me

TITLE=14-hydroxy-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=14-hydroxy-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=14-hydroxy-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=14-hydroxy-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=14-hydroxy-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=14-hydroxy-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=14-hydroxy-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=14-hydroxy-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=14-hydroxy-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=14-hydroxy-14-(hydroxymethyl)-5

TITLE=[(1S,3R,5R,6S,8S)-3-{[(2S,3R,4S,5S,6R)-6-[(benzoyloxy)methyl]-3,4,5-trihydroxyoxan-2-yl]oxy}-6-hydroxy-8-methyl-9,10-dioxatetracyclo[4.3.1.0�,?.0�,?]decan-2-yl]methyl benzoate

TITLE=[(1S,3R,5R,6S,8S)-3-{[(2S,3R,4S,5S,6R)-6-[(benzoyloxy)methyl]-3,4,5-trihydroxyoxan-2-yl]oxy}-6-hydroxy-8-methyl-9,10-dioxatetracyclo[4.3.1.0�,?.0�,?]decan-2-yl]methyl benzoate

TITLE=[(1S,3R,5R,6S,8S)-3-{[(2S,3R,4S,5S,6R)-6-[(benzoyloxy)methyl]-3,4,5-trihydroxyoxan-2-yl]oxy}-6-hydroxy-8-methyl-9,10-dioxatetracyclo[4.3.1.0�,?.0�,?]decan-2-yl]methyl benzoate

TITLE=[(1S,3R,5R,6S,8S)-3-{[(2S,3R,4S,5S,6R)-6-[(benzoyloxy)methyl]-3,4,5-trihydroxyoxan-2-yl]oxy}-6-hydroxy-8-methyl-9,10-dioxatetracyclo[4.3.1.0�,?.0�,?]decan-2-yl]methyl benzoate

TITLE=[(1S,3R,5R,6S,8S)-3-{[(2S,3R,4S,5S,6R)-6-[(benzoyloxy)methyl]-3,4,5-trihydroxyoxan-2-yl]oxy}-6-hydroxy-8-methyl-9,10-dioxatetracyclo[4.3.1.0�,?.0�,?]decan-2-yl]methyl benzoate

TITLE=[(1S,3R,5R,6S,8S)-3-{[(2S,3R,4S,5S,6R)-6-[(benzoyloxy)methyl]-3,4,5-trihydroxyo

...

TITLE=(2E)-6-{13,17-dihydroxy-7,7,12,16-tetramethyl-6-[(3,4,5-trihydroxyoxan-2-yl)oxy]pentacyclo[9.7.0.0�,�.0�,?.0��,�?]octadecan-15-yl}-2-methylhept-2-enoic acid

TITLE=(2S,3S,9R,12R,13R,14S,15R,16R,17S)-3,12,13,15,16-pentahydroxy-2,6,14,17-tetramethyl-10-oxatetracyclo[7.7.1.0�,?.0��,�?]heptadec-5-ene-4,11-dione

TITLE=(2S,3S,9R,12R,13R,14S,15R,16R,17S)-3,12,13,15,16-pentahydroxy-2,6,14,17-tetramethyl-10-oxatetracyclo[7.7.1.0�,?.0��,�?]heptadec-5-ene-4,11-dione

TITLE=(2S,3S,9R,12R,13R,14S,15R,16R,17S)-3,12,13,15,16-pentahydroxy-2,6,14,17-tetramethyl-10-oxatetracyclo[7.7.1.0�,?.0��,�?]heptadec-5-ene-4,11-dione

TITLE=(2S,3S,9R,12R,13R,14S,15R,16R,17S)-3,12,13,15,16-pentahydroxy-2,6,14,17-tetramethyl-10-oxatetracyclo[7.7.1.0�,?.0��,�?]heptadec-5-ene-4,11-dione

TITLE=(2S,3S,9R,12R,13R,14S,15R,16R,17S)-3,12,13,15,16-pentahydroxy-2,6,14,17-tetramethyl-10-oxatetracyclo[7.7.1.0�,?.0��,�?]heptadec-5-ene-4,11-dione

TITLE=(2S,3S,9R,12R,13R,14S,15R,16R,17S)-3,12,13,15,16-pentahydroxy-2,6,14,1

TITLE=(2R,7S,13R,14R,16S,19R,20S)-19-(furan-3-yl)-9,9,13,20-tetramethyl-5,12,17-trioxo-4,8,15,18-tetraoxahexacyclo[11.9.0.0�,?.0�,�?.0�?,�?.0�?,�?]docosan-11-yl acetate

TITLE=(2R,7S,13R,14R,16S,19R,20S)-19-(furan-3-yl)-9,9,13,20-tetramethyl-5,12,17-trioxo-4,8,15,18-tetraoxahexacyclo[11.9.0.0�,?.0�,�?.0�?,�?.0�?,�?]docosan-11-yl acetate

TITLE=(2R,7S,13R,14R,16S,19R,20S)-19-(furan-3-yl)-9,9,13,20-tetramethyl-5,12,17-trioxo-4,8,15,18-tetraoxahexacyclo[11.9.0.0�,?.0�,�?.0�?,�?.0�?,�?]docosan-11-yl acetate

TITLE=(2R,7S,13R,14R,16S,19R,20S)-19-(furan-3-yl)-9,9,13,20-tetramethyl-5,12,17-trioxo-4,8,15,18-tetraoxahexacyclo[11.9.0.0�,?.0�,�?.0�?,�?.0�?,�?]docosan-11-yl acetate

TITLE=(2R,7S,13R,14R,16S,19R,20S)-19-(furan-3-yl)-9,9,13,20-tetramethyl-5,12,17-trioxo-4,8,15,18-tetraoxahexacyclo[11.9.0.0�,?.0�,�?.0�?,�?.0�?,�?]docosan-11-yl acetate

TITLE=(2R,7S,13R,14R,16S,19R,20S)-19-(furan-3-yl)-9,9,13,20-tetramethyl-5,12,17-trioxo-4,8,15,18-tetraoxahexacyclo[11.9.0.0�,?.0�,�?.0�?,�?.0�?,�?]doc

TITLE=(1R,2S,4R,5S,6S,11R,12R,14S,16R)-5,6,11-trihydroxy-4-(hydroxymethyl)-8,12,15,15-tetramethyl-7-oxo-3-oxapentacyclo[9.5.0.0�,?.0?,�?.0�?,�?]hexadec-8-en-14-yl dodecanoate

TITLE=(1R,2S,4R,5S,6S,11R,12R,14S,16R)-5,6,11-trihydroxy-4-(hydroxymethyl)-8,12,15,15-tetramethyl-7-oxo-3-oxapentacyclo[9.5.0.0�,?.0?,�?.0�?,�?]hexadec-8-en-14-yl dodecanoate

TITLE=(1R,2S,4R,5S,6S,11R,12R,14S,16R)-5,6,11-trihydroxy-4-(hydroxymethyl)-8,12,15,15-tetramethyl-7-oxo-3-oxapentacyclo[9.5.0.0�,?.0?,�?.0�?,�?]hexadec-8-en-14-yl dodecanoate

TITLE=(1R,2S,4R,5S,6S,11R,12R,14S,16R)-5,6,11-trihydroxy-4-(hydroxymethyl)-8,12,15,15-tetramethyl-7-oxo-3-oxapentacyclo[9.5.0.0�,?.0?,�?.0�?,�?]hexadec-8-en-14-yl dodecanoate

TITLE=(1R,2S,4R,5S,6S,11R,12R,14S,16R)-5,6,11-trihydroxy-4-(hydroxymethyl)-8,12,15,15-tetramethyl-7-oxo-3-oxapentacyclo[9.5.0.0�,?.0?,�?.0�?,�?]hexadec-8-en-14-yl dodecanoate

TITLE=(1R,2S,4R,5S,6S,11R,12R,14S,16R)-5,6,11-trihydroxy-4-(hydroxymethyl)-8,12,15,15-tetramethyl-7-oxo-3-oxapentacyclo[

TITLE=(1S,2S,3R,4S,7R,10S,12R,15S)-4,12-bis(acetyloxy)-1,9,15-trihydroxy-10,14,17,17-tetramethyl-11-oxo-6-oxatetracyclo[11.3.1.0�,�?.0?,?]heptadec-13-en-2-yl benzoate

TITLE=(1S,2S,3R,4S,7R,10S,12R,15S)-4,12-bis(acetyloxy)-1,9,15-trihydroxy-10,14,17,17-tetramethyl-11-oxo-6-oxatetracyclo[11.3.1.0�,�?.0?,?]heptadec-13-en-2-yl benzoate

TITLE=(1S,2S,3R,4S,7R,10S,12R,15S)-4,12-bis(acetyloxy)-1,9,15-trihydroxy-10,14,17,17-tetramethyl-11-oxo-6-oxatetracyclo[11.3.1.0�,�?.0?,?]heptadec-13-en-2-yl benzoate

TITLE=(1S,2S,3R,4S,7R,10S,12R,15S)-4,12-bis(acetyloxy)-1,9,15-trihydroxy-10,14,17,17-tetramethyl-11-oxo-6-oxatetracyclo[11.3.1.0�,�?.0?,?]heptadec-13-en-2-yl benzoate

TITLE=(1S,2S,3R,4S,7R,10S,12R,15S)-4,12-bis(acetyloxy)-1,9,15-trihydroxy-10,14,17,17-tetramethyl-11-oxo-6-oxatetracyclo[11.3.1.0�,�?.0?,?]heptadec-13-en-2-yl benzoate

TITLE=(1S,2S,3R,4S,7R,10S,12R,15S)-4,12-bis(acetyloxy)-1,9,15-trihydroxy-10,14,17,17-tetramethyl-11-oxo-6-oxatetracyclo[11.3.1.0�,�?.0?,?]heptadec-13-en-2-yl be

TITLE=(15S,16Z,17S)-16-ethylidene-4-hydroxy-15-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-2,10,14,20-tetraoxatetracyclo[21.2.2.1�,?.0��,�?]octacosa-1(25),3(28),4,6,12,23,26-heptaene-11,19-dione

TITLE=(15S,16Z,17S)-16-ethylidene-4-hydroxy-15-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-2,10,14,20-tetraoxatetracyclo[21.2.2.1�,?.0��,�?]octacosa-1(25),3(28),4,6,12,23,26-heptaene-11,19-dione

TITLE=(15S,16Z,17S)-16-ethylidene-4-hydroxy-15-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-2,10,14,20-tetraoxatetracyclo[21.2.2.1�,?.0��,�?]octacosa-1(25),3(28),4,6,12,23,26-heptaene-11,19-dione

TITLE=methyl (2R)-2-[(1S,2R,4S,9R,10R,14S,15S,17S)-17-(acetyloxy)-9-(furan-3-yl)-1-hydroxy-10,14,16,16-tetramethyl-7,18-dioxo-3,8-dioxapentacyclo[12.3.1.0�,?.0?,��.0?,�?]octadecan-15-yl]-2-hydroxyacetate

TITLE=methyl (2R)-2-[(1S,2R,4S,9R,10R,14S,15S,17S)-17-(acetyloxy)-9-(furan-3-yl)-1-hydroxy-10,14,16,16-tetramethyl-7,18-dioxo-3,8-dioxa

In [44]:
file_mgf = os.path.join(PATH_MS_DATA, 'allGNPS_negative_smiles_191106.mgf')
with open(file_mgf, 'w') as f:
    for ID in IDs_negative_smiles:
        spec = collected_spectra[ID]
        for l in spec:
            try:
                f.write(l.encode('utf8', 'replace').decode('utf8', 'replace'))
            except UnicodeEncodeError:
                f.write(l.encode('ascii', 'replace').decode('utf8', 'replace'))
                print(l)
        f.write('\n')

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=11,22-dimethyl (6E,17E)-3,14-dioxo-8,19-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})-4,9,15,20-tetraoxatricyclo[16.4.0.0?,��]docosa-6,10,17,21-tetraene-11,22-dicarboxylate

TITLE=11,22-dimethyl (6E,17E)-3,14-dioxo-8,19-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})-4,9,15,20-tetraoxatricyclo[16.4.0.0?,��]docosa-6,10,17,21-tetraene-11,22-dicarboxylate

TITLE=2,4,18-trihydroxy-15-(3-hydroxy-3-methylbutyl)-7,9,13-trioxapentacyclo[10.8.0.0�,�?.0?,?.0�?,�?]icosa-1(12),2,5,10,14,16,18-heptaen-20-one

TITLE=2,4,18-trihydroxy-15-(3-hydroxy-3-methylbutyl)-7,9,13-trioxapentacyclo[10.8.0.0�,�

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=11,22-dimethyl (6E,17E)-3,14-dioxo-8,19-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})-4,9,15,20-tetraoxatricyclo[16.4.0.0?,��]docosa-6,10,17,21-tetraene-11,22-dicarboxylate

TITLE=11,22-dimethyl (6E,17E)-3,14-dioxo-8,19-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})-4,9,15,20-tetraoxatricyclo[16.4.0.0?,��]docosa-6,10,17,21-tetraene-11,22-dicarboxylate

TITLE=11,22-dimethyl (6E,17E)-3,14-dioxo-8,19-bis({[(2S,3R

TITLE=15-formyl-16-hydroxy-1,2,14,17,17-pentamethyl-8-(prop-1-en-2-yl)pentacyclo[11.7.0.0�,�?.0?,?.0�?,�?]icosane-5-carboxylic acid

TITLE=15-formyl-16-hydroxy-1,2,14,17,17-pentamethyl-8-(prop-1-en-2-yl)pentacyclo[11.7.0.0�,�?.0?,?.0�?,�?]icosane-5-carboxylic acid

TITLE=15-formyl-16-hydroxy-1,2,14,17,17-pentamethyl-8-(prop-1-en-2-yl)pentacyclo[11.7.0.0�,�?.0?,?.0�?,�?]icosane-5-carboxylic acid

TITLE=[(1S,3R,5R,6S,8S)-3-{[(2S,3R,4S,5S,6R)-6-[(benzoyloxy)methyl]-3,4,5-trihydroxyoxan-2-yl]oxy}-6-hydroxy-8-methyl-9,10-dioxatetracyclo[4.3.1.0�,?.0�,?]decan-2-yl]methyl benzoate

TITLE=[(1S,3R,5R,6S,8S)-3-{[(2S,3R,4S,5S,6R)-6-[(benzoyloxy)methyl]-3,4,5-trihydroxyoxan-2-yl]oxy}-6-hydroxy-8-methyl-9,10-dioxatetracyclo[4.3.1.0�,?.0�,?]decan-2-yl]methyl benzoate

TITLE=[(1S,3R,5R,6S,8S)-3-{[(2S,3R,4S,5S,6R)-6-[(benzoyloxy)methyl]-3,4,5-trihydroxyoxan-2-yl]oxy}-6-hydroxy-8-methyl-9,10-dioxatetracyclo[4.3.1.0�,?.0�,?]decan-2-yl]methyl benzoate

TITLE=[(1S,2S,3R,5R,6S,8S)-6-hydroxy-8-methyl-3-{[(2

TITLE=(1R,21S,23R)-6,7,8,11,12,13,22,23-octahydroxy-3,16-dioxo-2,17,20-trioxatetracyclo[17.3.1.0?,?.0�?,�?]tricosa-4(9),5,7,10(15),11,13-hexaen-21-yl 3,4,5-trihydroxybenzoate

TITLE=(1R,21S,23R)-6,7,8,11,12,13,22,23-octahydroxy-3,16-dioxo-2,17,20-trioxatetracyclo[17.3.1.0?,?.0�?,�?]tricosa-4(9),5,7,10(15),11,13-hexaen-21-yl 3,4,5-trihydroxybenzoate

TITLE=(1R,21S,23R)-6,7,8,11,12,13,22,23-octahydroxy-3,16-dioxo-2,17,20-trioxatetracyclo[17.3.1.0?,?.0�?,�?]tricosa-4(9),5,7,10(15),11,13-hexaen-21-yl 3,4,5-trihydroxybenzoate

TITLE=methyl (1S,5R,9S,13R)-5,9-dimethyl-14-methylidenetetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylate

TITLE=methyl (1S,5R,9S,13R)-5,9-dimethyl-14-methylidenetetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylate

TITLE=methyl (1S,5R,9S,13R)-5,9-dimethyl-14-methylidenetetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylate

TITLE=2,4-dihydroxy-5-methoxy-1'',6'',8''-trimethyldispiro[bis(oxolane)-3,2':5',7''-[3]oxatricyclo[6.3.1.0?,��]dodecane]-2''-one

TITLE=2,4-di

TITLE=6,18-bis(1-hydroxy-3-methylbut-2-en-1-yl)-11,22-dimethyl-5,10,17,21-tetraoxaheptacyclo[11.7.2.0�,?.0�,��.0?,?.0�?,�?.0�?,�?]docosa-8,14(20)-diene-3,7,15,19-tetrone

TITLE=6,18-bis(1-hydroxy-3-methylbut-2-en-1-yl)-11,22-dimethyl-5,10,17,21-tetraoxaheptacyclo[11.7.2.0�,?.0�,��.0?,?.0�?,�?.0�?,�?]docosa-8,14(20)-diene-3,7,15,19-tetrone

TITLE=6,18-bis(1-hydroxy-3-methylbut-2-en-1-yl)-11,22-dimethyl-5,10,17,21-tetraoxaheptacyclo[11.7.2.0�,?.0�,��.0?,?.0�?,�?.0�?,�?]docosa-8,14(20)-diene-3,7,15,19-tetrone

TITLE=(2S,3R,5S,9R)-2,3-dihydroxy-14-(hydroxymethyl)-5,9-dimethyl-14-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}tetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=(2S,3R,5S,9R)-2,3-dihydroxy-14-(hydroxymethyl)-5,9-dimethyl-14-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}tetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=(2S,3R,5S,9R)-2,3-dihydroxy-14-(hydroxymethyl)-5,9-dimethyl-14-{[(2S,3R,4S,5S,6R)-3,4,5-tri

TITLE=(3S,5R,8S)-2-[(benzoyloxy)methyl]-8-methyl-3-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-9,10-dioxatetracyclo[4.3.1.0�,?.0�,?]decane-6-sulfonic acid

TITLE=(3S,5R,8S)-2-[(benzoyloxy)methyl]-8-methyl-3-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-9,10-dioxatetracyclo[4.3.1.0�,?.0�,?]decane-6-sulfonic acid

TITLE=(3S,5R,8S)-2-[(benzoyloxy)methyl]-8-methyl-3-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-9,10-dioxatetracyclo[4.3.1.0�,?.0�,?]decane-6-sulfonic acid

TITLE=(2S,3S,4S,8R,9S,13R,14R,15S,16R)-3,4,8,14,15-pentahydroxy-2,13,16-trimethyl-6-methylidene-10-oxatetracyclo[7.6.1.0�,?.0��,�?]hexadecan-11-one

TITLE=(2S,3S,4S,8R,9S,13R,14R,15S,16R)-3,4,8,14,15-pentahydroxy-2,13,16-trimethyl-6-methylidene-10-oxatetracyclo[7.6.1.0�,?.0��,�?]hexadecan-11-one

TITLE=(2S,3S,4S,8R,9S,13R,14R,15S,16R)-3,4,8,14,15-pentahydroxy-2,13,16-trimethyl-6-methylidene-10-oxatetracyclo[7.6.1.0�,?.0��,�?]hexadecan-11-one

TITLE=(2E,4E)

TITLE=(2R,3R,6R,8R,9S,12S,13R,14R,15R,16R)-6,8,14,15-tetrahydroxy-2,6,13,16-tetramethyl-3-{[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-10-oxatetracyclo[7.6.1.0�,?.0��,�?]hexadec-4-en-11-one

TITLE=(2R,3R,6R,8R,9S,12S,13R,14R,15R,16R)-6,8,14,15-tetrahydroxy-2,6,13,16-tetramethyl-3-{[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-10-oxatetracyclo[7.6.1.0�,?.0��,�?]hexadec-4-en-11-one

TITLE=(2R,3R,6R,8R,9S,12S,13R,14R,15R,16R)-6,8,14,15-tetrahydroxy-2,6,13,16-tetramethyl-3-{[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-10-oxatetracyclo[7.6.1.0�,?.0��,�?]hexadec-4-en-11-one

TITLE=13-methoxy-1,6-diazatetracyclo[7.6.1.0?,�?.0�?,�?]hexadeca-3,5,7,9(16),10,12,14-heptaen-2-one

TITLE=13-methoxy-1,6-diazatetracyclo[7.6.1.0?,�?.0�?,�?]hexadeca-3,5,7,9(16),10,12,14-heptaen-2-one

TITLE=13-methoxy-1,6-diazatetracyclo[7.6.1.0?,�?.0�?,�?]hexadeca-3,5,7,9(16),10,12,14-heptaen-2-one

TITLE=4-[(7R,11S,14S,16R)-14-[(4-methoxy-6-methyl-5-{[(

TITLE=(5S,9R)-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=(5S,9R)-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=(5S,9R)-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=(15S,16Z,17S)-16-ethylidene-4-hydroxy-15-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-2,10,14,20-tetraoxatetracyclo[21.2.2.1�,?.0��,�?]octacosa-1(25),3(28),4,6,12,23,26-heptaene-11,19-dione

TITLE=(15S,16Z,17S)-16-ethylidene-4-hydroxy-15-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-2,10,14,20-tetraoxatetracyclo[21.2.2.1�,?.0��,�?]octacosa-1(25),3(28),4,6,12,23,26-heptaene-11,19-dione

TITLE=(15S,16Z,17S)-16-ethylidene-4-hydroxy-15-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-2,10,14,20-tetraoxatetracyclo[21.2.2.1�,?.0��,�?]octacosa-1(25),3(28),4,6,12,23,26-heptaene-11,19-dione

TITLE=methyl (2R)-2-[(1S,2R,